In [14]:
import pandas as pd
import seaborn as sns
import os
import numpy as np


In [15]:
ingresos_mmpp = pd.read_csv('../Procesos_Ingresos_2021.csv')


# Transformaciones

### Unidad de medida pesajes

In [16]:
# Kilos a toneladas
ingresos_mmpp['Valor'] = [ingresos_mmpp.Valor.iloc[i]/1000 if ingresos_mmpp['Unidad de medida'].iloc[i] == 'Kilos' else ingresos_mmpp.Valor.iloc[i]  for i in ingresos_mmpp.index]
ingresos_mmpp['Unidad de medida'] = ingresos_mmpp['Unidad de medida'].apply(lambda u: 'Ton' if u == 'Kilos' else u)

### Agrupación indicadores

In [17]:
ingresos_mmpp.Indicador.unique()

dict_indicadores_tipo = {
    'Ingreso Materia Bruta Prima Propia':'MP Bruta',
    'Ingreso Materia Prima Neta o Pagada Propia':'MP Neta',
    'Unidades por kilo mmpp propia*': "No aplica",
    'Ingreso Materia Prima Bruta Terceros':'MP Bruta',
    'Ingreso Materia Prima Neta o Pagada Terceros':'MP Neta',
    'Unidades por kilo mmpp terceros*': "No aplica", 
    'Ingreso Materia Prima Propia':'MP Bruta',
    'Ingreso Materia Prima Neta Propia':'MP Neta',
    'Ingreso Materia Prima Terceros':'MP Bruta',
    'Ingreso Materia Prima Neta Terceros':'MP Neta',
    'Ingreso Materia Prima Bruta Propia':'MP Bruta'
}

ingresos_mmpp['Tipo'] = ingresos_mmpp.Indicador.apply(lambda i: dict_indicadores_tipo[i])

In [18]:
dict_indicadores_propiedad = {
    'Ingreso Materia Bruta Prima Propia':'MPB Propia',
    'Ingreso Materia Prima Neta o Pagada Propia':'MPN Propia',
    'Unidades por kilo mmpp propia*': "No aplica",
    'Ingreso Materia Prima Bruta Terceros':'MPB Terceros',
    'Ingreso Materia Prima Neta o Pagada Terceros':'MPN Terceros',
    'Unidades por kilo mmpp terceros*': "No aplica", 
    'Ingreso Materia Prima Propia':'MPB Propia',
    'Ingreso Materia Prima Neta Propia':'MPN Propia',
    'Ingreso Materia Prima Terceros':'MPB Terceros',
    'Ingreso Materia Prima Neta Terceros':'MPN Terceros',
    'Ingreso Materia Prima Bruta Propia':'MPB Propia'
}

ingresos_mmpp['Propiedad'] = ingresos_mmpp.Indicador.apply(lambda i: dict_indicadores_propiedad[i])

# Resumen Mensual Ingreso Materia Prima

### Resumen Mensual Ingreso Materia Prima Industria

#### Empresa 

In [19]:
dict_meses = {
    1:'Enero',
    2:'Febrero',
    3:'Marzo',
    4:'Abril',
    5:'Mayo',
    6:'Junio',
    7:'Julio',
    8:'Agosto',
    9:'Septiembre',
    10:'Octubre',
    11:'Noviembre',
    12:'Diciembre'
}

In [20]:
ingresos_mmpp.Empresa.unique()

array(['SA Chonchi', 'SA Quemchi', 'Landes', 'Camanchaca', 'Manuelita',
       'Sudmaris', 'Blue Shell'], dtype=object)

In [21]:
empresa = 'Manuelita'

In [22]:
pv = ingresos_mmpp[ingresos_mmpp.Empresa == empresa].pivot_table(index=['Tipo','Propiedad'], columns='Mes', values='Valor', aggfunc=sum)
pv = pv[pv.index.get_level_values('Tipo').isin(['MP Bruta', 'MP Neta'])]
total = pv.sum(axis=1).round(0)
average = pv.mean(axis=1).round(0)
pv[13] = total.copy()
pv[14] = average.copy()

pv2 = ingresos_mmpp[ingresos_mmpp.Empresa == empresa].pivot_table(index=['Tipo','Propiedad'], columns='Mes', values='Valor')
pv2 = pv2.groupby(level=0).transform(lambda x: (x / x.sum()).round(2))*100
pv2 = pv2[pv2.index.get_level_values('Tipo').isin(['MP Bruta', 'MP Neta'])]
total2 = pv2.mean(axis=1).round(0)
average2 = pv2.mean(axis=1).round(0)
pv2[13] = total2
pv2[14] = average2

largo = len(pv2.columns)
new_index = []
for i in range(1, largo+1):
    if i == largo - 1:
        new_index.append(('Total','Ton'))
        new_index.append(('Total','%'))
    elif i == largo:
        new_index.append(('Promedio','Ton'))
        new_index.append(('Promedio','%'))
    else: 
        new_index.append((dict_meses[i],'Ton'))
        new_index.append((dict_meses[i],'%'))

pv3 = pd.concat([pv, pv2], axis=1).sort_index(level=0,axis=1)
pv3.columns = pd.MultiIndex.from_tuples(new_index)
pv3

pv_empresa = pd.concat([
    d.append(d.sum().rename((k, 'Total')))
    for k, d in pv3.groupby(level=0)
]).append(pv3.sum().rename(('General', 'Total'))).round()

pv_empresa

Enero        Febrero          Marzo          Abril  \
                          Ton      %     Ton      %     Ton      %     Ton   
Tipo     Propiedad                                                           
MP Bruta MPB Propia    1812.0   59.0  2083.0   68.0  3114.0   82.0  1964.0   
         MPB Terceros  1261.0   41.0   997.0   32.0   685.0   18.0  1253.0   
         Total         3074.0  100.0  3080.0  100.0  3799.0  100.0  3217.0   
MP Neta  MPN Propia    1620.0   59.0  1897.0   68.0  2781.0   82.0  1727.0   
         MPN Terceros  1123.0   41.0   885.0   32.0   618.0   18.0  1084.0   
         Total         2743.0  100.0  2782.0  100.0  3399.0  100.0  2811.0   
General  Total         5817.0  200.0  5862.0  200.0  7198.0  200.0  6028.0   

                                Mayo          Junio         Julio         \
                           %     Ton      %     Ton      %    Ton      %   
Tipo     Propiedad                                                         
MP Bruta MPB Propia     61.0  2117.0   63.0  1591.0   57.0  380.0   79.0   
         MPB Terceros   39.0  1238.0   37.0  1196.0   43.0  102.0   21.0   
         Total         100.0  3355.0  100.0  2787.0  100.0  483.0  100.0   
MP Neta  MPN Propia     61.0  1797.0   64.0  1348.0   57.0  333.0   79.0   
         MPN Terceros   39.0  1016.0   36.0  1022.0   43.0   88.0   21.0   
         Total         100.0  2813.0  100.0  2370.0  100.0  421.0  100.0   
General  Total         200.0  6168.0  200.0  5157.0  200.0  904.0  200.0   

                         Total        Promedio         
                           Ton      %      Ton      %  
Tipo     Propiedad                                     
MP Bruta MPB Propia    13062.0   67.0   1866.0   67.0  
         MPB Terceros   6732.0   33.0    962.0   33.0  
         Total         19794.0  100.0   2828.0  100.0  
MP Neta  MPN Propia    11502.0   67.0   1643.0   67.0  
         MPN Terceros   5837.0   33.0    834.0   33.0  
         Total         17339.0  100.0   2477.0  100.0  
General  Total         37133.0  200.0   5305.0  200.0

In [23]:
pv_empresa2 = pv_empresa.reset_index(level=None, drop=False)

In [24]:
pv_empresa2

Tipo     Propiedad   Enero        Febrero          Marzo         \
                              Ton      %     Ton      %     Ton      %   
0  MP Bruta    MPB Propia  1812.0   59.0  2083.0   68.0  3114.0   82.0   
1  MP Bruta  MPB Terceros  1261.0   41.0   997.0   32.0   685.0   18.0   
2  MP Bruta         Total  3074.0  100.0  3080.0  100.0  3799.0  100.0   
3   MP Neta    MPN Propia  1620.0   59.0  1897.0   68.0  2781.0   82.0   
4   MP Neta  MPN Terceros  1123.0   41.0   885.0   32.0   618.0   18.0   
5   MP Neta         Total  2743.0  100.0  2782.0  100.0  3399.0  100.0   
6   General         Total  5817.0  200.0  5862.0  200.0  7198.0  200.0   

    Abril           Mayo          Junio         Julio           Total         \
      Ton      %     Ton      %     Ton      %    Ton      %      Ton      %   
0  1964.0   61.0  2117.0   63.0  1591.0   57.0  380.0   79.0  13062.0   67.0   
1  1253.0   39.0  1238.0   37.0  1196.0   43.0  102.0   21.0   6732.0   33.0   
2  3217.0  100.0  3355.0  100.0  2787.0  100.0  483.0  100.0  19794.0  100.0   
3  1727.0   61.0  1797.0   64.0  1348.0   57.0  333.0   79.0  11502.0   67.0   
4  1084.0   39.0  1016.0   36.0  1022.0   43.0   88.0   21.0   5837.0   33.0   
5  2811.0  100.0  2813.0  100.0  2370.0  100.0  421.0  100.0  17339.0  100.0   
6  6028.0  200.0  6168.0  200.0  5157.0  200.0  904.0  200.0  37133.0  200.0   

  Promedio         
       Ton      %  
0   1866.0   67.0  
1    962.0   33.0  
2   2828.0  100.0  
3   1643.0   67.0  
4    834.0   33.0  
5   2477.0  100.0  
6   5305.0  200.0

In [25]:
pv_empresa2.Tipo = ["" if pv_empresa2.Tipo.iloc[i] == pv_empresa2.Tipo.iloc[i-1] else pv_empresa2.Tipo.iloc[i] for i in range(len(pv_empresa2.Tipo))]

In [26]:
pv.empresa.Tipo.apply(lambda t: )

SyntaxError: invalid syntax (Temp/ipykernel_15932/150569337.py, line 1)

In [27]:
largo = len(pv_empresa.columns)
for i in range(1,largo+1,2):
    if pv_empresa.iloc[0,i]+pv_empresa.iloc[1,i] != 100:
        pv_empresa.iloc[:,[i-1, i]] = pv_empresa.iloc[:,[i,i-1]].copy() 
        
pv_empresa

Enero        Febrero          Marzo          Abril  \
                          Ton      %     Ton      %     Ton      %     Ton   
Tipo     Propiedad                                                           
MP Bruta MPB Propia    1812.0   59.0  2083.0   68.0  3114.0   82.0  1964.0   
         MPB Terceros  1261.0   41.0   997.0   32.0   685.0   18.0  1253.0   
         Total         3074.0  100.0  3080.0  100.0  3799.0  100.0  3217.0   
MP Neta  MPN Propia    1620.0   59.0  1897.0   68.0  2781.0   82.0  1727.0   
         MPN Terceros  1123.0   41.0   885.0   32.0   618.0   18.0  1084.0   
         Total         2743.0  100.0  2782.0  100.0  3399.0  100.0  2811.0   
General  Total         5817.0  200.0  5862.0  200.0  7198.0  200.0  6028.0   

                                Mayo          Junio         Julio         \
                           %     Ton      %     Ton      %    Ton      %   
Tipo     Propiedad                                                         
MP Bruta MPB Propia     61.0  2117.0   63.0  1591.0   57.0  380.0   79.0   
         MPB Terceros   39.0  1238.0   37.0  1196.0   43.0  102.0   21.0   
         Total         100.0  3355.0  100.0  2787.0  100.0  483.0  100.0   
MP Neta  MPN Propia     61.0  1797.0   64.0  1348.0   57.0  333.0   79.0   
         MPN Terceros   39.0  1016.0   36.0  1022.0   43.0   88.0   21.0   
         Total         100.0  2813.0  100.0  2370.0  100.0  421.0  100.0   
General  Total         200.0  6168.0  200.0  5157.0  200.0  904.0  200.0   

                         Total        Promedio         
                           Ton      %      Ton      %  
Tipo     Propiedad                                     
MP Bruta MPB Propia    13062.0   67.0   1866.0   67.0  
         MPB Terceros   6732.0   33.0    962.0   33.0  
         Total         19794.0  100.0   2828.0  100.0  
MP Neta  MPN Propia    11502.0   67.0   1643.0   67.0  
         MPN Terceros   5837.0   33.0    834.0   33.0  
         Total         17339.0  100.0   2477.0  100.0  
General  Total         37133.0  200.0   5305.0  200.0

#### Todas las industrias

In [28]:
pv = ingresos_mmpp.pivot_table(index=['Tipo','Propiedad'], columns='Mes', values='Valor', aggfunc=sum)
pv = pv[pv.index.get_level_values('Tipo').isin(['MP Bruta', 'MP Neta'])]
total = pv.sum(axis=1).round(0)
average = pv.mean(axis=1).round(0)
pv[13] = total.copy()
pv[14] = average.copy()

pv2 = ingresos_mmpp.pivot_table(index=['Tipo','Propiedad'], columns='Mes', values='Valor')
pv2 = pv2.groupby(level=0).transform(lambda x: (x / x.sum()).round(2))*100
pv2 = pv2[pv2.index.get_level_values('Tipo').isin(['MP Bruta', 'MP Neta'])]
total2 = pv2.mean(axis=1).round(0)
average2 = pv2.mean(axis=1).round(0)
pv2[13] = total2
pv2[14] = average2

largo = len(pv2.columns)
new_index = []
for i in range(1, largo+1):
    if i == largo - 1:
        new_index.append(('Total','Ton'))
        new_index.append(('Total','%'))
    elif i == largo:
        new_index.append(('Promedio','Ton'))
        new_index.append(('Promedio','%'))
    else: 
        new_index.append((dict_meses[i],'Ton'))
        new_index.append((dict_meses[i],'%'))

pv3 = pd.concat([pv, pv2], axis=1).sort_index(level=0,axis=1)
pv3.columns = pd.MultiIndex.from_tuples(new_index)

pv_todas = pd.concat([
    d.append(d.sum().rename((k, 'Total')))
    for k, d in pv3.groupby(level=0)
]).append(pv3.sum().rename(('General', 'Total'))).round()

pv_todas


Enero         Febrero           Marzo         \
                           Ton      %      Ton      %      Ton      %   
Tipo     Propiedad                                                      
MP Bruta MPB Propia    14590.0   53.0  17519.0   60.0  18839.0   65.0   
         MPB Terceros  12943.0   47.0  11613.0   40.0  11740.0   35.0   
         Total         27533.0  100.0  29132.0  100.0  30578.0  100.0   
MP Neta  MPN Propia    12723.0   54.0  14531.0   60.0  18829.0   65.0   
         MPN Terceros  10977.0   46.0   9822.0   40.0  10188.0   35.0   
         Total         23700.0  100.0  24353.0  100.0  29017.0  100.0   
General  Total         51234.0  200.0  53485.0  200.0  59596.0  200.0   

                         Abril            Mayo         ...    Julio         \
                           Ton      %      Ton      %  ...      Ton      %   
Tipo     Propiedad                                     ...                   
MP Bruta MPB Propia    13147.0   58.0  17059.0   63.0  ...   9032.0   42.0   
         MPB Terceros  11199.0   42.0  11812.0   37.0  ...  14830.0   58.0   
         Total         24346.0  100.0  28871.0  100.0  ...  23862.0  100.0   
MP Neta  MPN Propia    13876.0   59.0  18182.0   64.0  ...  10870.0   46.0   
         MPN Terceros   9677.0   41.0  10223.0   36.0  ...  12568.0   54.0   
         Total         23554.0  100.0  28405.0  100.0  ...  23438.0  100.0   
General  Total         47900.0  200.0  57276.0  200.0  ...  47300.0  200.0   

                        Agosto        Septiembre          Total            \
                           Ton      %        Ton       %    Ton         %   
Tipo     Propiedad                                                          
MP Bruta MPB Propia     4707.0   29.0       46.0  1426.0   52.0  109954.0   
         MPB Terceros  11770.0   71.0       54.0  1648.0   48.0  101245.0   
         Total         16477.0  100.0      100.0  3074.0  100.0  211199.0   
MP Neta  MPN Propia     4052.0   29.0       46.0  1214.0   53.0  109049.0   
         MPN Terceros   9815.0   71.0       54.0  1398.0   47.0   86229.0   
         Total         13867.0  100.0      100.0  2612.0  100.0  195278.0   
General  Total         30344.0  200.0      200.0  5686.0  200.0  406477.0   

                      Promedio         
                           Ton      %  
Tipo     Propiedad                     
MP Bruta MPB Propia    12217.0   52.0  
         MPB Terceros  11249.0   48.0  
         Total         23466.0  100.0  
MP Neta  MPN Propia    12117.0   53.0  
         MPN Terceros   9581.0   47.0  
         Total         21698.0  100.0  
General  Total         45164.0  200.0  

[7 rows x 22 columns]

In [29]:
largo = len(pv_todas.columns)
for i in range(1,largo+1,2):
    if pv_todas.iloc[0,i]+pv_todas.iloc[1,i] != 100:
        pv_todas.iloc[:,[i-1, i]] = pv_todas.iloc[:,[i,i-1]].copy() 
        
pv_todas

Enero         Febrero           Marzo         \
                           Ton      %      Ton      %      Ton      %   
Tipo     Propiedad                                                      
MP Bruta MPB Propia    14590.0   53.0  17519.0   60.0  18839.0   65.0   
         MPB Terceros  12943.0   47.0  11613.0   40.0  11740.0   35.0   
         Total         27533.0  100.0  29132.0  100.0  30578.0  100.0   
MP Neta  MPN Propia    12723.0   54.0  14531.0   60.0  18829.0   65.0   
         MPN Terceros  10977.0   46.0   9822.0   40.0  10188.0   35.0   
         Total         23700.0  100.0  24353.0  100.0  29017.0  100.0   
General  Total         51234.0  200.0  53485.0  200.0  59596.0  200.0   

                         Abril            Mayo         ...    Julio         \
                           Ton      %      Ton      %  ...      Ton      %   
Tipo     Propiedad                                     ...                   
MP Bruta MPB Propia    13147.0   58.0  17059.0   63.0  ...   9032.0   42.0   
         MPB Terceros  11199.0   42.0  11812.0   37.0  ...  14830.0   58.0   
         Total         24346.0  100.0  28871.0  100.0  ...  23862.0  100.0   
MP Neta  MPN Propia    13876.0   59.0  18182.0   64.0  ...  10870.0   46.0   
         MPN Terceros   9677.0   41.0  10223.0   36.0  ...  12568.0   54.0   
         Total         23554.0  100.0  28405.0  100.0  ...  23438.0  100.0   
General  Total         47900.0  200.0  57276.0  200.0  ...  47300.0  200.0   

                        Agosto        Septiembre            Total         \
                           Ton      %        Ton      %       Ton      %   
Tipo     Propiedad                                                         
MP Bruta MPB Propia     4707.0   29.0     1426.0   46.0  109954.0   52.0   
         MPB Terceros  11770.0   71.0     1648.0   54.0  101245.0   48.0   
         Total         16477.0  100.0     3074.0  100.0  211199.0  100.0   
MP Neta  MPN Propia     4052.0   29.0     1214.0   46.0  109049.0   53.0   
         MPN Terceros   9815.0   71.0     1398.0   54.0   86229.0   47.0   
         Total         13867.0  100.0     2612.0  100.0  195278.0  100.0   
General  Total         30344.0  200.0     5686.0  200.0  406477.0  200.0   

                      Promedio         
                           Ton      %  
Tipo     Propiedad                     
MP Bruta MPB Propia    12217.0   52.0  
         MPB Terceros  11249.0   48.0  
         Total         23466.0  100.0  
MP Neta  MPN Propia    12117.0   53.0  
         MPN Terceros   9581.0   47.0  
         Total         21698.0  100.0  
General  Total         45164.0  200.0  

[7 rows x 22 columns]

In [30]:
pv_todas

Enero         Febrero           Marzo         \
                           Ton      %      Ton      %      Ton      %   
Tipo     Propiedad                                                      
MP Bruta MPB Propia    14590.0   53.0  17519.0   60.0  18839.0   65.0   
         MPB Terceros  12943.0   47.0  11613.0   40.0  11740.0   35.0   
         Total         27533.0  100.0  29132.0  100.0  30578.0  100.0   
MP Neta  MPN Propia    12723.0   54.0  14531.0   60.0  18829.0   65.0   
         MPN Terceros  10977.0   46.0   9822.0   40.0  10188.0   35.0   
         Total         23700.0  100.0  24353.0  100.0  29017.0  100.0   
General  Total         51234.0  200.0  53485.0  200.0  59596.0  200.0   

                         Abril            Mayo         ...    Julio         \
                           Ton      %      Ton      %  ...      Ton      %   
Tipo     Propiedad                                     ...                   
MP Bruta MPB Propia    13147.0   58.0  17059.0   63.0  ...   9032.0   42.0   
         MPB Terceros  11199.0   42.0  11812.0   37.0  ...  14830.0   58.0   
         Total         24346.0  100.0  28871.0  100.0  ...  23862.0  100.0   
MP Neta  MPN Propia    13876.0   59.0  18182.0   64.0  ...  10870.0   46.0   
         MPN Terceros   9677.0   41.0  10223.0   36.0  ...  12568.0   54.0   
         Total         23554.0  100.0  28405.0  100.0  ...  23438.0  100.0   
General  Total         47900.0  200.0  57276.0  200.0  ...  47300.0  200.0   

                        Agosto        Septiembre            Total         \
                           Ton      %        Ton      %       Ton      %   
Tipo     Propiedad                                                         
MP Bruta MPB Propia     4707.0   29.0     1426.0   46.0  109954.0   52.0   
         MPB Terceros  11770.0   71.0     1648.0   54.0  101245.0   48.0   
         Total         16477.0  100.0     3074.0  100.0  211199.0  100.0   
MP Neta  MPN Propia     4052.0   29.0     1214.0   46.0  109049.0   53.0   
         MPN Terceros   9815.0   71.0     1398.0   54.0   86229.0   47.0   
         Total         13867.0  100.0     2612.0  100.0  195278.0  100.0   
General  Total         30344.0  200.0     5686.0  200.0  406477.0  200.0   

                      Promedio         
                           Ton      %  
Tipo     Propiedad                     
MP Bruta MPB Propia    12217.0   52.0  
         MPB Terceros  11249.0   48.0  
         Total         23466.0  100.0  
MP Neta  MPN Propia    12117.0   53.0  
         MPN Terceros   9581.0   47.0  
         Total         21698.0  100.0  
General  Total         45164.0  200.0  

[7 rows x 22 columns]

#### Otras industrias

In [31]:
pv = ingresos_mmpp[ingresos_mmpp.Empresa != empresa].pivot_table(index=['Tipo','Propiedad'], columns='Mes', values='Valor', aggfunc=sum)
pv = pv[pv.index.get_level_values('Tipo').isin(['MP Bruta', 'MP Neta'])]
total = pv.sum(axis=1).round(0)
average = pv.mean(axis=1).round(0)
pv[13] = total.copy()
pv[14] = average.copy()

pv2 = ingresos_mmpp[ingresos_mmpp.Empresa != empresa].pivot_table(index=['Tipo','Propiedad'], columns='Mes', values='Valor', aggfunc=sum)
pv2 = pv2.groupby(level=0).transform(lambda x: (x / x.sum()).round(2))*100
pv2 = pv2[pv2.index.get_level_values('Tipo').isin(['MP Bruta', 'MP Neta'])]
total2 = pv2.mean(axis=1).round(0)
average2 = pv2.mean(axis=1).round(0)
pv2[13] = total2
pv2[14] = average2

largo = len(pv2.columns)
new_index = []
for i in range(1, largo+1):
    if i == largo - 1:
        new_index.append(('Total','Ton'))
        new_index.append(('Total','%'))
    elif i == largo:
        new_index.append(('Promedio','Ton'))
        new_index.append(('Promedio','%'))
    else: 
        new_index.append((dict_meses[i],'Ton'))
        new_index.append((dict_meses[i],'%'))

pv3 = pd.concat([pv, pv2], axis=1).sort_index(level=0,axis=1)
pv3.columns = pd.MultiIndex.from_tuples(new_index)
pv3

pv_otras = pd.concat([
    d.append(d.sum().rename((k, 'Total')))
    for k, d in pv3.groupby(level=0)
]).append(pv3.sum().rename(('General', 'Total'))).round()

largo = len(pv_otras.columns)
for i in range(1,largo+1,2):
    if pv_otras.iloc[0,i]+pv_otras.iloc[1,i] != 100:
        pv_otras.iloc[:,[i-1, i]] = pv_otras.iloc[:,[i,i-1]].copy() 
        
pv_otras

Enero         Febrero           Marzo         \
                           Ton      %      Ton      %      Ton      %   
Tipo     Propiedad                                                      
MP Bruta MPB Propia    12778.0   52.0  15436.0   59.0  15725.0   59.0   
         MPB Terceros  11682.0   48.0  10616.0   41.0  11055.0   41.0   
         Total         24460.0  100.0  26052.0  100.0  26780.0  100.0   
MP Neta  MPN Propia    11103.0   53.0  12635.0   59.0  16049.0   63.0   
         MPN Terceros   9854.0   47.0   8936.0   41.0   9570.0   37.0   
         Total         20957.0  100.0  21571.0  100.0  25619.0  100.0   
General  Total         45416.0  200.0  47623.0  200.0  52398.0  200.0   

                         Abril            Mayo         ...    Julio         \
                           Ton      %      Ton      %  ...      Ton      %   
Tipo     Propiedad                                     ...                   
MP Bruta MPB Propia    13147.0   54.0  15095.0   59.0  ...   9032.0   38.0   
         MPB Terceros  11199.0   46.0  10559.0   41.0  ...  14830.0   62.0   
         Total         24346.0  100.0  25654.0  100.0  ...  23862.0  100.0   
MP Neta  MPN Propia    13876.0   59.0  16455.0   64.0  ...  10870.0   46.0   
         MPN Terceros   9677.0   41.0   9139.0   36.0  ...  12568.0   54.0   
         Total         23554.0  100.0  25594.0  100.0  ...  23438.0  100.0   
General  Total         47900.0  200.0  51248.0  200.0  ...  47300.0  200.0   

                        Agosto        Septiembre            Total         \
                           Ton      %        Ton      %       Ton      %   
Tipo     Propiedad                                                         
MP Bruta MPB Propia     3116.0   23.0     1046.0   40.0   96891.0   48.0   
         MPB Terceros  10574.0   77.0     1545.0   60.0   94513.0   52.0   
         Total         13690.0  100.0     2591.0  100.0  191404.0  100.0   
MP Neta  MPN Propia     2704.0   24.0      881.0   40.0   97547.0   51.0   
         MPN Terceros   8793.0   76.0     1310.0   60.0   80391.0   49.0   
         Total         11497.0  100.0     2191.0  100.0  177938.0  100.0   
General  Total         25186.0  200.0     4783.0  200.0  369342.0  200.0   

                      Promedio         
                           Ton      %  
Tipo     Propiedad                     
MP Bruta MPB Propia    10766.0   48.0  
         MPB Terceros  10501.0   52.0  
         Total         21267.0  100.0  
MP Neta  MPN Propia    10839.0   51.0  
         MPN Terceros   8932.0   49.0  
         Total         19771.0  100.0  
General  Total         41038.0  200.0  

[7 rows x 22 columns]

In [32]:
largo = len(pv_otras.columns)
for i in range(1,largo+1,2):
    if pv_otras.iloc[0,i]+pv_otras.iloc[1,i] != 100:
        pv_otras.iloc[:,[i-1, i]] = pv_otras.iloc[:,[i,i-1]].copy() 
        
pv_otras

Enero         Febrero           Marzo         \
                           Ton      %      Ton      %      Ton      %   
Tipo     Propiedad                                                      
MP Bruta MPB Propia    12778.0   52.0  15436.0   59.0  15725.0   59.0   
         MPB Terceros  11682.0   48.0  10616.0   41.0  11055.0   41.0   
         Total         24460.0  100.0  26052.0  100.0  26780.0  100.0   
MP Neta  MPN Propia    11103.0   53.0  12635.0   59.0  16049.0   63.0   
         MPN Terceros   9854.0   47.0   8936.0   41.0   9570.0   37.0   
         Total         20957.0  100.0  21571.0  100.0  25619.0  100.0   
General  Total         45416.0  200.0  47623.0  200.0  52398.0  200.0   

                         Abril            Mayo         ...    Julio         \
                           Ton      %      Ton      %  ...      Ton      %   
Tipo     Propiedad                                     ...                   
MP Bruta MPB Propia    13147.0   54.0  15095.0   59.0  ...   9032.0   38.0   
         MPB Terceros  11199.0   46.0  10559.0   41.0  ...  14830.0   62.0   
         Total         24346.0  100.0  25654.0  100.0  ...  23862.0  100.0   
MP Neta  MPN Propia    13876.0   59.0  16455.0   64.0  ...  10870.0   46.0   
         MPN Terceros   9677.0   41.0   9139.0   36.0  ...  12568.0   54.0   
         Total         23554.0  100.0  25594.0  100.0  ...  23438.0  100.0   
General  Total         47900.0  200.0  51248.0  200.0  ...  47300.0  200.0   

                        Agosto        Septiembre            Total         \
                           Ton      %        Ton      %       Ton      %   
Tipo     Propiedad                                                         
MP Bruta MPB Propia     3116.0   23.0     1046.0   40.0   96891.0   48.0   
         MPB Terceros  10574.0   77.0     1545.0   60.0   94513.0   52.0   
         Total         13690.0  100.0     2591.0  100.0  191404.0  100.0   
MP Neta  MPN Propia     2704.0   24.0      881.0   40.0   97547.0   51.0   
         MPN Terceros   8793.0   76.0     1310.0   60.0   80391.0   49.0   
         Total         11497.0  100.0     2191.0  100.0  177938.0  100.0   
General  Total         25186.0  200.0     4783.0  200.0  369342.0  200.0   

                      Promedio         
                           Ton      %  
Tipo     Propiedad                     
MP Bruta MPB Propia    10766.0   48.0  
         MPB Terceros  10501.0   52.0  
         Total         21267.0  100.0  
MP Neta  MPN Propia    10839.0   51.0  
         MPN Terceros   8932.0   49.0  
         Total         19771.0  100.0  
General  Total         41038.0  200.0  

[7 rows x 22 columns]

## Ingreso Materia Prima Detalle
#### Acumulado Año Industria

In [33]:
dict_propiedad = {
 'Ingreso Materia Bruta Prima Propia':'Materia Prima Propia',
 'Ingreso Materia Prima Neta o Pagada Propia':'Materia Prima Propia',
 'Unidades por kilo mmpp propia*':'Materia Prima Propia',
 'Ingreso Materia Prima Bruta Terceros':'Materia Prima Terceros',
 'Ingreso Materia Prima Neta o Pagada Terceros':'Materia Prima Terceros',
 'Unidades por kilo mmpp terceros*':'Materia Prima Terceros',
 'Ingreso Materia Prima Propia':'Materia Prima Propia',
 'Ingreso Materia Prima Neta Propia':'Materia Prima Propia',
 'Ingreso Materia Prima Terceros':'Materia Prima Terceros',
 'Ingreso Materia Prima Neta Terceros':'Materia Prima Terceros',
 'Ingreso Materia Prima Bruta Propia':'Materia Prima Propia',
 '% Rechazo Materia Prima Propia':'Materia Prima Propia',
 '% Rechazo Materia Prima Terceros':'Materia Prima Terceros',
 '% Rechazo Materia Prima Total':'Total Materia Prima'
}

dict_indicadores_2 = {
 'Ingreso Materia Bruta Prima Propia':'Ingreso Materia Prima Bruta',
 'Ingreso Materia Prima Neta o Pagada Propia':'Ingreso Materia Prima Neta',
 'Unidades por kilo mmpp propia*':'Unidades por kilo',
 'Ingreso Materia Prima Bruta Terceros':'Ingreso Materia Prima Bruta',
 'Ingreso Materia Prima Neta o Pagada Terceros':'Ingreso Materia Prima Neta',
 'Unidades por kilo mmpp terceros*':'Unidades por kilo',
 'Ingreso Materia Prima Propia':'Ingreso Materia Prima Bruta',
 'Ingreso Materia Prima Neta Propia':'Ingreso Materia Prima Neta',
 'Ingreso Materia Prima Terceros':'Ingreso Materia Prima Bruta',
 'Ingreso Materia Prima Neta Terceros':'Ingreso Materia Prima Neta',
 'Ingreso Materia Prima Bruta Propia':'Ingreso Materia Prima Bruta',
 '% Rechazo Materia Prima Propia':'% Rechazo MMPP',
 '% Rechazo Materia Prima Terceros':'% Rechazo MMPP',
 '% Rechazo Materia Prima Total':'% Rechazo MMPP'
}

In [34]:
df = ingresos_mmpp.copy()
df['Indicadores'] = [dict_indicadores_2[df.Indicador.iloc[i]] for i in range(len(df.Indicador))]
df['Propietario'] = [dict_propiedad[df.Indicador.iloc[i]] for i in range(len(df.Indicador))]

In [35]:
new_df = pd.DataFrame(columns = df.columns)

for e in df.Empresa.unique():
    for m in df.Mes.unique():
        new_row = {
            'Indicador':'Ingreso Materia Prima Bruta', 
            'Valor': df[(df.Empresa == e) & (df.Mes == m) & (df.Tipo == 'MP Bruta')].Valor.sum(),
            'Unidad de medida':'Ton', 
            'Definición':'', 
            'Mes':m, 
            'Año':2021,
            'Empresa':e,
            'Tipo':'MP Bruta', 
            'Propiedad':'', 
            'Indicadores':'Ingreso Materia Prima Bruta', 
            'Propietario':'Total Materia Prima'
        }
        new_df = new_df.append(new_row, ignore_index = True)
        
        new_row = {
            'Indicador':'Ingreso Materia Prima Neta', 
            'Valor': df[(df.Empresa == e) & (df.Mes == m) & (df.Tipo == 'MP Neta')].Valor.sum(),
            'Unidad de medida':'Ton', 
            'Definición':'', 
            'Mes':m, 
            'Año':2021,
            'Empresa':e,
            'Tipo':'MP Bruta', 
            'Propiedad':'', 
            'Indicadores':'Ingreso Materia Prima Neta', 
            'Propietario':'Total Materia Prima'
        }
        new_df = new_df.append(new_row, ignore_index = True)
        

In [36]:
df = df.append(new_df, ignore_index=True)
df = df.groupby(['Propietario','Indicadores']).Valor.agg(['sum', np.mean], axis='columns')
df = df.round(2)
df.columns = ['Total', 'Promedio']

In [37]:
df.head(5)

Total  Promedio
Propietario            Indicadores                                     
Materia Prima Propia   Ingreso Materia Prima Bruta  109953.55   2036.18
                       Ingreso Materia Prima Neta   109048.77   1848.28
                       Unidades por kilo              4093.80     63.97
Materia Prima Terceros Ingreso Materia Prima Bruta  101244.81   1716.01
                       Ingreso Materia Prima Neta    86228.55   1461.50

In [38]:
# Promedio ponderado unidades por kilo
df2 = ingresos_mmpp.copy()
df2['Indicadores'] = [dict_indicadores_2[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
df2['Propietario'] = [dict_propiedad[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
filtro = (df2.Indicadores == 'Ingreso Materia Prima Neta') | (df2.Indicadores=='Unidades por kilo')

df3 = df2[filtro].groupby(['Empresa','Mes','Propietario','Indicadores']).Valor.sum().unstack(level=-1).fillna(0)

df3_propia = df3.loc[idx[:,:,'Materia Prima Propia'],:]
uxk_propia = (df3_propia['Ingreso Materia Prima Neta']*df3_propia['Unidades por kilo']).sum()/df3_propia['Ingreso Materia Prima Neta'].sum()


df3_terceros = df3.loc[idx[:,:,'Materia Prima Terceros'],:]
uxk_terceros = (df3_terceros['Ingreso Materia Prima Neta']*df3_terceros['Unidades por kilo']).sum()/df3_terceros['Ingreso Materia Prima Neta'].sum()

print(uxk_propia, uxk_terceros)

NameError: name 'idx' is not defined

In [39]:
for i, j in df.index:
    df.loc[(i,'% Rechazo mmpp'), 'Total'] = (df.loc[(i,'Ingreso Materia Prima Bruta'), 'Total']-df.loc[(i,'Ingreso Materia Prima Neta'), 'Total'])*100/df.loc[(i,'Ingreso Materia Prima Bruta'), 'Total']
    if i != 'Total Materia Prima':
        df.loc[(i,'% MP Bruta Propia sobre Total MP Bruta'),'Total'] = df.loc[(i,'Ingreso Materia Prima Bruta'), 'Total']/df.loc[('Total Materia Prima','Ingreso Materia Prima Bruta'),'Total']*100
    if (i == 'Materia Prima Propia') & (j=='Unidades por kilo'):
        # Promedio ponderado unidades por kilo
        df2 = ingresos_mmpp.copy()
        df2['Indicadores'] = [dict_indicadores_2[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        df2['Propietario'] = [dict_propiedad[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        filtro = (df2.Indicadores == 'Ingreso Materia Prima Neta') | (df2.Indicadores=='Unidades por kilo')
        df3 = df2[filtro].groupby(['Empresa','Mes','Propietario','Indicadores']).Valor.sum().unstack(level=-1).fillna(0)
        df3_propia = df3.loc[idx[:,:,'Materia Prima Propia'],:]
        uxk_propia = (df3_propia['Ingreso Materia Prima Neta']*df3_propia['Unidades por kilo']).sum()/df3_propia['Ingreso Materia Prima Neta'].sum()
        df.loc[(i,j),'Total'] = uxk_propia    
        
    if (i == 'Materia Prima Terceros') & (j=='Unidades por kilo'):
        # Promedio ponderado unidades por kilo
        df2 = ingresos_mmpp.copy()
        df2['Indicadores'] = [dict_indicadores_2[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        df2['Propietario'] = [dict_propiedad[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        filtro = (df2.Indicadores == 'Ingreso Materia Prima Neta') | (df2.Indicadores=='Unidades por kilo')
        df3 = df2[filtro].groupby(['Empresa','Mes','Propietario','Indicadores']).Valor.sum().unstack(level=-1).fillna(0)
        df3_terceros = df3.loc[idx[:,:,'Materia Prima Terceros'],:]
        uxk_terceros = (df3_terceros['Ingreso Materia Prima Neta']*df3_terceros['Unidades por kilo']).sum()/df3_terceros['Ingreso Materia Prima Neta'].sum()
        df.loc[(i,j),'Total'] = uxk_terceros

df.round(2).sort_index(ascending = [True, False])

NameError: name 'idx' is not defined

In [40]:
list(df.index)

[('Materia Prima Propia', 'Ingreso Materia Prima Bruta'),
 ('Materia Prima Propia', 'Ingreso Materia Prima Neta'),
 ('Materia Prima Propia', 'Unidades por kilo'),
 ('Materia Prima Terceros', 'Ingreso Materia Prima Bruta'),
 ('Materia Prima Terceros', 'Ingreso Materia Prima Neta'),
 ('Materia Prima Terceros', 'Unidades por kilo'),
 ('Total Materia Prima', 'Ingreso Materia Prima Bruta'),
 ('Total Materia Prima', 'Ingreso Materia Prima Neta'),
 ('Materia Prima Propia', '% Rechazo mmpp'),
 ('Materia Prima Propia', '% MP Bruta Propia sobre Total MP Bruta')]

In [41]:
df = df.loc[[('Materia Prima Propia', 'Ingreso Materia Prima Bruta'),
 ('Materia Prima Propia', 'Ingreso Materia Prima Neta'),
 ('Materia Prima Propia', 'Unidades por kilo'),
 ('Materia Prima Propia', '% Rechazo mmpp'),
 ('Materia Prima Propia', '% MP Bruta Propia sobre Total MP Bruta'),
 ('Materia Prima Terceros', 'Ingreso Materia Prima Bruta'),
 ('Materia Prima Terceros', 'Ingreso Materia Prima Neta'),
 ('Materia Prima Terceros', 'Unidades por kilo'),
 ('Materia Prima Terceros', '% Rechazo mmpp'),
 ('Materia Prima Terceros', '% MP Bruta Propia sobre Total MP Bruta'),
 ('Total Materia Prima', 'Ingreso Materia Prima Bruta'),
 ('Total Materia Prima', 'Ingreso Materia Prima Neta'),
 ('Total Materia Prima', '% Rechazo mmpp'),
 ],:].fillna("")

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: MultiIndex([('Materia Prima Terceros', ...),\n            ('Materia Prima Terceros', ...),\n            (   'Total Materia Prima', ...)],\n           names=['Propietario', 'Indicadores']). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [42]:
df

Total  \
Propietario            Indicadores                                             
Materia Prima Propia   Ingreso Materia Prima Bruta             109953.550000   
                       Ingreso Materia Prima Neta              109048.770000   
                       Unidades por kilo                         4093.800000   
Materia Prima Terceros Ingreso Materia Prima Bruta             101244.810000   
                       Ingreso Materia Prima Neta               86228.550000   
                       Unidades por kilo                         3279.320000   
Total Materia Prima    Ingreso Materia Prima Bruta             211198.350000   
                       Ingreso Materia Prima Neta              195277.320000   
Materia Prima Propia   % Rechazo mmpp                               0.822875   
                       % MP Bruta Propia sobre Total MP Bruta      52.061747   

                                                               Promedio  
Propietario            Indicadores                                       
Materia Prima Propia   Ingreso Materia Prima Bruta              2036.18  
                       Ingreso Materia Prima Neta               1848.28  
                       Unidades por kilo                          63.97  
Materia Prima Terceros Ingreso Materia Prima Bruta              1716.01  
                       Ingreso Materia Prima Neta               1461.50  
                       Unidades por kilo                          59.62  
Total Materia Prima    Ingreso Materia Prima Bruta              3352.35  
                       Ingreso Materia Prima Neta               3099.64  
Materia Prima Propia   % Rechazo mmpp                               NaN  
                       % MP Bruta Propia sobre Total MP Bruta       NaN

#### Acumulado Año Empresa particular

In [44]:
Empresa = 'Sudmaris'


#### Acumulado Año Otras Industrias

In [45]:
Empresa = 'Sudmaris'

# Nuevas columnas
df = ingresos_mmpp[ingresos_mmpp.Empresa != empresa].copy()
df['Indicadores'] = [dict_indicadores_2[df.Indicador.iloc[i]] for i in range(len(df.Indicador))]
df['Propietario'] = [dict_propiedad[df.Indicador.iloc[i]] for i in range(len(df.Indicador))]

# Añadir nuevas columnas para el total
new_df = pd.DataFrame(columns = df.columns)

for e in df.Empresa.unique():
    for m in df.Mes.unique():
        new_row = {
            'Indicador':'Ingreso Materia Prima Bruta', 
            'Valor': df[(df.Empresa == e) & (df.Mes == m) & (df.Tipo == 'MP Bruta')].Valor.sum(),
            'Unidad de medida':'Ton', 
            'Definición':'', 
            'Mes':m, 
            'Año':2021,
            'Empresa':e,
            'Tipo':'MP Bruta', 
            'Propiedad':'', 
            'Indicadores':'Ingreso Materia Prima Bruta', 
            'Propietario':'Total Materia Prima'
        }
        new_df = new_df.append(new_row, ignore_index = True)
        
        new_row = {
            'Indicador':'Ingreso Materia Prima Neta', 
            'Valor': df[(df.Empresa == e) & (df.Mes == m) & (df.Tipo == 'MP Neta')].Valor.sum(),
            'Unidad de medida':'Ton', 
            'Definición':'', 
            'Mes':m, 
            'Año':2021,
            'Empresa':e,
            'Tipo':'MP Bruta', 
            'Propiedad':'', 
            'Indicadores':'Ingreso Materia Prima Neta', 
            'Propietario':'Total Materia Prima'
        }
        new_df = new_df.append(new_row, ignore_index = True)
        
df = df.append(new_df, ignore_index=True)
df = df.groupby(['Propietario','Indicadores']).Valor.agg(['sum', np.mean], axis='columns')
df = df.round(2)
df.columns = ['Total', 'Promedio']

# Promedio ponderado unidades por kilo
df2 = ingresos_mmpp[ingresos_mmpp.Empresa != empresa].copy()
df2['Indicadores'] = [dict_indicadores_2[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
df2['Propietario'] = [dict_propiedad[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
filtro = (df2.Indicadores == 'Ingreso Materia Prima Neta') | (df2.Indicadores=='Unidades por kilo')

df3 = df2[filtro].groupby(['Empresa','Mes','Propietario','Indicadores']).Valor.sum().unstack(level=-1).fillna(0)

df3_propia = df3.loc[idx[:,:,'Materia Prima Propia'],:]
uxk_propia = (df3_propia['Ingreso Materia Prima Neta']*df3_propia['Unidades por kilo']).sum()/df3_propia['Ingreso Materia Prima Neta'].sum()


df3_terceros = df3.loc[idx[:,:,'Materia Prima Terceros'],:]
uxk_terceros = (df3_terceros['Ingreso Materia Prima Neta']*df3_terceros['Unidades por kilo']).sum()/df3_terceros['Ingreso Materia Prima Neta'].sum()


#Cambios en Unidades por Kilo y %MP Bruta

for i, j in df.index:
    df.loc[(i,'% Rechazo mmpp'), 'Total'] = (df.loc[(i,'Ingreso Materia Prima Bruta'), 'Total']-df.loc[(i,'Ingreso Materia Prima Neta'), 'Total'])*100/df.loc[(i,'Ingreso Materia Prima Bruta'), 'Total']
    if i != 'Total Materia Prima':
        df.loc[(i,'% MP Bruta Propia sobre Total MP Bruta'),'Total'] = df.loc[(i,'Ingreso Materia Prima Bruta'), 'Total']/df.loc[('Total Materia Prima','Ingreso Materia Prima Bruta'),'Total']*100
    if (i == 'Materia Prima Propia') & (j=='Unidades por kilo'):
        # Promedio ponderado unidades por kilo
        df2 = ingresos_mmpp.copy()
        df2['Indicadores'] = [dict_indicadores_2[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        df2['Propietario'] = [dict_propiedad[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        filtro = (df2.Indicadores == 'Ingreso Materia Prima Neta') | (df2.Indicadores=='Unidades por kilo')
        df3 = df2[filtro].groupby(['Empresa','Mes','Propietario','Indicadores']).Valor.sum().unstack(level=-1).fillna(0)
        df3_propia = df3.loc[idx[:,:,'Materia Prima Propia'],:]
        uxk_propia = (df3_propia['Ingreso Materia Prima Neta']*df3_propia['Unidades por kilo']).sum()/df3_propia['Ingreso Materia Prima Neta'].sum()
        df.loc[(i,j),'Total'] = uxk_propia    
        
    if (i == 'Materia Prima Terceros') & (j=='Unidades por kilo'):
        # Promedio ponderado unidades por kilo
        df2 = ingresos_mmpp.copy()
        df2['Indicadores'] = [dict_indicadores_2[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        df2['Propietario'] = [dict_propiedad[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        filtro = (df2.Indicadores == 'Ingreso Materia Prima Neta') | (df2.Indicadores=='Unidades por kilo')
        df3 = df2[filtro].groupby(['Empresa','Mes','Propietario','Indicadores']).Valor.sum().unstack(level=-1).fillna(0)
        df3_terceros = df3.loc[idx[:,:,'Materia Prima Terceros'],:]
        uxk_terceros = (df3_terceros['Ingreso Materia Prima Neta']*df3_terceros['Unidades por kilo']).sum()/df3_terceros['Ingreso Materia Prima Neta'].sum()
        df.loc[(i,j),'Total'] = uxk_terceros

df.round(2).sort_index(ascending = [True, False])


df = df.loc[[('Materia Prima Propia', 'Ingreso Materia Prima Bruta'),
 ('Materia Prima Propia', 'Ingreso Materia Prima Neta'),
 ('Materia Prima Propia', 'Unidades por kilo'),
 ('Materia Prima Propia', '% Rechazo mmpp'),
 ('Materia Prima Propia', '% MP Bruta Propia sobre Total MP Bruta'),
 ('Materia Prima Terceros', 'Ingreso Materia Prima Bruta'),
 ('Materia Prima Terceros', 'Ingreso Materia Prima Neta'),
 ('Materia Prima Terceros', 'Unidades por kilo'),
 ('Materia Prima Terceros', '% Rechazo mmpp'),
 ('Materia Prima Terceros', '% MP Bruta Propia sobre Total MP Bruta'),
 ('Total Materia Prima', 'Ingreso Materia Prima Bruta'),
 ('Total Materia Prima', 'Ingreso Materia Prima Neta'),
 ('Total Materia Prima', '% Rechazo mmpp'),
 ],:].fillna("")

df.round(2)

NameError: name 'idx' is not defined

#### Mes Empresas

In [ ]:
mes = 9

# Nuevas columnas
df = ingresos_mmpp[ingresos_mmpp.Mes == mes].copy()
df['Indicadores'] = [dict_indicadores_2[df.Indicador.iloc[i]] for i in range(len(df.Indicador))]
df['Propietario'] = [dict_propiedad[df.Indicador.iloc[i]] for i in range(len(df.Indicador))]

# Añadir nuevas columnas para el total
new_df = pd.DataFrame(columns = df.columns)

for e in df.Empresa.unique():
    for m in df.Mes.unique():
        new_row = {
            'Indicador':'Ingreso Materia Prima Bruta', 
            'Valor': df[(df.Empresa == e) & (df.Mes == m) & (df.Tipo == 'MP Bruta')].Valor.sum(),
            'Unidad de medida':'Ton', 
            'Definición':'', 
            'Mes':m, 
            'Año':2021,
            'Empresa':e,
            'Tipo':'MP Bruta', 
            'Propiedad':'', 
            'Indicadores':'Ingreso Materia Prima Bruta', 
            'Propietario':'Total Materia Prima'
        }
        new_df = new_df.append(new_row, ignore_index = True)
        
        new_row = {
            'Indicador':'Ingreso Materia Prima Neta', 
            'Valor': df[(df.Empresa == e) & (df.Mes == m) & (df.Tipo == 'MP Neta')].Valor.sum(),
            'Unidad de medida':'Ton', 
            'Definición':'', 
            'Mes':m, 
            'Año':2021,
            'Empresa':e,
            'Tipo':'MP Bruta', 
            'Propiedad':'', 
            'Indicadores':'Ingreso Materia Prima Neta', 
            'Propietario':'Total Materia Prima'
        }
        new_df = new_df.append(new_row, ignore_index = True)
        
df = df.append(new_df, ignore_index=True)
df = df.groupby(['Propietario','Indicadores']).Valor.agg(['sum', np.mean], axis='columns')
df = df.round(2)
df.columns = ['Total', 'Promedio']

# Promedio ponderado unidades por kilo
df2 = ingresos_mmpp[ingresos_mmpp.Mes == mes].copy()
df2['Indicadores'] = [dict_indicadores_2[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
df2['Propietario'] = [dict_propiedad[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
filtro = (df2.Indicadores == 'Ingreso Materia Prima Neta') | (df2.Indicadores=='Unidades por kilo')

df3 = df2[filtro].groupby(['Empresa','Mes','Propietario','Indicadores']).Valor.sum().unstack(level=-1).fillna(0)

df3_propia = df3.loc[idx[:,:,'Materia Prima Propia'],:]
uxk_propia = (df3_propia['Ingreso Materia Prima Neta']*df3_propia['Unidades por kilo']).sum()/df3_propia['Ingreso Materia Prima Neta'].sum()


df3_terceros = df3.loc[idx[:,:,'Materia Prima Terceros'],:]
uxk_terceros = (df3_terceros['Ingreso Materia Prima Neta']*df3_terceros['Unidades por kilo']).sum()/df3_terceros['Ingreso Materia Prima Neta'].sum()


#Cambios en Unidades por Kilo y %MP Bruta

for i, j in df.index:
    df.loc[(i,'% Rechazo mmpp'), 'Total'] = (df.loc[(i,'Ingreso Materia Prima Bruta'), 'Total']-df.loc[(i,'Ingreso Materia Prima Neta'), 'Total'])*100/df.loc[(i,'Ingreso Materia Prima Bruta'), 'Total']
    if i != 'Total Materia Prima':
        df.loc[(i,'% MP Bruta Propia sobre Total MP Bruta'),'Total'] = df.loc[(i,'Ingreso Materia Prima Bruta'), 'Total']/df.loc[('Total Materia Prima','Ingreso Materia Prima Bruta'),'Total']*100
    if (i == 'Materia Prima Propia') & (j=='Unidades por kilo'):
        # Promedio ponderado unidades por kilo
        df2 = ingresos_mmpp.copy()
        df2['Indicadores'] = [dict_indicadores_2[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        df2['Propietario'] = [dict_propiedad[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        filtro = (df2.Indicadores == 'Ingreso Materia Prima Neta') | (df2.Indicadores=='Unidades por kilo')
        df3 = df2[filtro].groupby(['Empresa','Mes','Propietario','Indicadores']).Valor.sum().unstack(level=-1).fillna(0)
        df3_propia = df3.loc[idx[:,:,'Materia Prima Propia'],:]
        uxk_propia = (df3_propia['Ingreso Materia Prima Neta']*df3_propia['Unidades por kilo']).sum()/df3_propia['Ingreso Materia Prima Neta'].sum()
        df.loc[(i,j),'Total'] = uxk_propia    
        
    if (i == 'Materia Prima Terceros') & (j=='Unidades por kilo'):
        # Promedio ponderado unidades por kilo
        df2 = ingresos_mmpp.copy()
        df2['Indicadores'] = [dict_indicadores_2[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        df2['Propietario'] = [dict_propiedad[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        filtro = (df2.Indicadores == 'Ingreso Materia Prima Neta') | (df2.Indicadores=='Unidades por kilo')
        df3 = df2[filtro].groupby(['Empresa','Mes','Propietario','Indicadores']).Valor.sum().unstack(level=-1).fillna(0)
        df3_terceros = df3.loc[idx[:,:,'Materia Prima Terceros'],:]
        uxk_terceros = (df3_terceros['Ingreso Materia Prima Neta']*df3_terceros['Unidades por kilo']).sum()/df3_terceros['Ingreso Materia Prima Neta'].sum()
        df.loc[(i,j),'Total'] = uxk_terceros

df.round(2).sort_index(ascending = [True, False])


df = df.loc[[('Materia Prima Propia', 'Ingreso Materia Prima Bruta'),
 ('Materia Prima Propia', 'Ingreso Materia Prima Neta'),
 ('Materia Prima Propia', 'Unidades por kilo'),
 ('Materia Prima Propia', '% Rechazo mmpp'),
 ('Materia Prima Propia', '% MP Bruta Propia sobre Total MP Bruta'),
 ('Materia Prima Terceros', 'Ingreso Materia Prima Bruta'),
 ('Materia Prima Terceros', 'Ingreso Materia Prima Neta'),
 ('Materia Prima Terceros', 'Unidades por kilo'),
 ('Materia Prima Terceros', '% Rechazo mmpp'),
 ('Materia Prima Terceros', '% MP Bruta Propia sobre Total MP Bruta'),
 ('Total Materia Prima', 'Ingreso Materia Prima Bruta'),
 ('Total Materia Prima', 'Ingreso Materia Prima Neta'),
 ('Total Materia Prima', '% Rechazo mmpp'),
 ],:].fillna("")

df.round(2)

#### Mes Empresa particular

In [46]:
df = ingresos_mmpp[(ingresos_mmpp.Empresa == empresa) & (ingresos_mmpp.Mes == mes)].copy()
df['Indicadores'] = [dict_indicadores_2[df.Indicador.iloc[i]] for i in range(len(df.Indicador))]
df['Propietario'] = [dict_propiedad[df.Indicador.iloc[i]] for i in range(len(df.Indicador))]
df

NameError: name 'mes' is not defined

In [47]:
Empresa = 'Sudmaris'
mes = 9

# Nuevas columnas
df = ingresos_mmpp[(ingresos_mmpp.Empresa == empresa) & (ingresos_mmpp.Mes == mes)].copy()
df['Indicadores'] = [dict_indicadores_2[df.Indicador.iloc[i]] for i in range(len(df.Indicador))]
df['Propietario'] = [dict_propiedad[df.Indicador.iloc[i]] for i in range(len(df.Indicador))]
df.fillna(0)

# Añadir nuevas columnas para el total
new_df = pd.DataFrame(columns = df.columns)

for e in df.Empresa.unique():
    for m in df.Mes.unique():
        new_row = {
            'Indicador':'Ingreso Materia Prima Bruta', 
            'Valor': df[(df.Empresa == e) & (df.Mes == m) & (df.Tipo == 'MP Bruta')].Valor.sum(),
            'Unidad de medida':'Ton', 
            'Definición':'', 
            'Mes':m, 
            'Año':2021,
            'Empresa':e,
            'Tipo':'MP Bruta', 
            'Propiedad':'', 
            'Indicadores':'Ingreso Materia Prima Bruta', 
            'Propietario':'Total Materia Prima'
        }
        new_df = new_df.append(new_row, ignore_index = True)
        
        new_row = {
            'Indicador':'Ingreso Materia Prima Neta', 
            'Valor': df[(df.Empresa == e) & (df.Mes == m) & (df.Tipo == 'MP Neta')].Valor.sum(),
            'Unidad de medida':'Ton', 
            'Definición':'', 
            'Mes':m, 
            'Año':2021,
            'Empresa':e,
            'Tipo':'MP Bruta', 
            'Propiedad':'', 
            'Indicadores':'Ingreso Materia Prima Neta', 
            'Propietario':'Total Materia Prima'
        }
        new_df = new_df.append(new_row, ignore_index = True)
        
df = df.append(new_df, ignore_index=True)
df = df.groupby(['Propietario','Indicadores']).Valor.agg(['sum', np.mean], axis='columns')
df = df.round(2)
df.columns = ['Total', 'Promedio']

# Promedio ponderado unidades por kilo
df2 = ingresos_mmpp[(ingresos_mmpp.Empresa == empresa) & (ingresos_mmpp.Mes == mes)].copy()
df2['Indicadores'] = [dict_indicadores_2[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
df2['Propietario'] = [dict_propiedad[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
filtro = (df2.Indicadores == 'Ingreso Materia Prima Neta') | (df2.Indicadores=='Unidades por kilo')

df3 = df2[filtro].groupby(['Empresa','Mes','Propietario','Indicadores']).Valor.sum().unstack(level=-1).fillna(0)

df3_propia = df3.loc[idx[:,:,'Materia Prima Propia'],:]
uxk_propia = (df3_propia['Ingreso Materia Prima Neta']*df3_propia['Unidades por kilo']).sum()/df3_propia['Ingreso Materia Prima Neta'].sum()


df3_terceros = df3.loc[idx[:,:,'Materia Prima Terceros'],:]
uxk_terceros = (df3_terceros['Ingreso Materia Prima Neta']*df3_terceros['Unidades por kilo']).sum()/df3_terceros['Ingreso Materia Prima Neta'].sum()


#Cambios en Unidades por Kilo y %MP Bruta

for i, j in df.index:
    df.loc[(i,'% Rechazo mmpp'), 'Total'] = (df.loc[(i,'Ingreso Materia Prima Bruta'), 'Total']-df.loc[(i,'Ingreso Materia Prima Neta'), 'Total'])*100/df.loc[(i,'Ingreso Materia Prima Bruta'), 'Total']
    if i != 'Total Materia Prima':
        df.loc[(i,'% MP Bruta Propia sobre Total MP Bruta'),'Total'] = df.loc[(i,'Ingreso Materia Prima Bruta'), 'Total']/df.loc[('Total Materia Prima','Ingreso Materia Prima Bruta'),'Total']*100
    if (i == 'Materia Prima Propia') & (j=='Unidades por kilo'):
        # Promedio ponderado unidades por kilo
        df2 = ingresos_mmpp.copy()
        df2['Indicadores'] = [dict_indicadores_2[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        df2['Propietario'] = [dict_propiedad[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        filtro = (df2.Indicadores == 'Ingreso Materia Prima Neta') | (df2.Indicadores=='Unidades por kilo')
        df3 = df2[filtro].groupby(['Empresa','Mes','Propietario','Indicadores']).Valor.sum().unstack(level=-1).fillna(0)
        df3_propia = df3.loc[idx[:,:,'Materia Prima Propia'],:]
        uxk_propia = (df3_propia['Ingreso Materia Prima Neta']*df3_propia['Unidades por kilo']).sum()/df3_propia['Ingreso Materia Prima Neta'].sum()
        df.loc[(i,j),'Total'] = uxk_propia    
        
    if (i == 'Materia Prima Terceros') & (j=='Unidades por kilo'):
        # Promedio ponderado unidades por kilo
        df2 = ingresos_mmpp.copy()
        df2['Indicadores'] = [dict_indicadores_2[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        df2['Propietario'] = [dict_propiedad[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        filtro = (df2.Indicadores == 'Ingreso Materia Prima Neta') | (df2.Indicadores=='Unidades por kilo')
        df3 = df2[filtro].groupby(['Empresa','Mes','Propietario','Indicadores']).Valor.sum().unstack(level=-1).fillna(0)
        df3_terceros = df3.loc[idx[:,:,'Materia Prima Terceros'],:]
        uxk_terceros = (df3_terceros['Ingreso Materia Prima Neta']*df3_terceros['Unidades por kilo']).sum()/df3_terceros['Ingreso Materia Prima Neta'].sum()
        df.loc[(i,j),'Total'] = uxk_terceros

df.round(2).sort_index(ascending = [True, False])

try: 
    df = df.loc[[('Materia Prima Propia', 'Ingreso Materia Prima Bruta'),
     ('Materia Prima Propia', 'Ingreso Materia Prima Neta'),
     ('Materia Prima Propia', 'Unidades por kilo'),
     ('Materia Prima Propia', '% Rechazo mmpp'),
     ('Materia Prima Propia', '% MP Bruta Propia sobre Total MP Bruta'),
     ('Materia Prima Terceros', 'Ingreso Materia Prima Bruta'),
     ('Materia Prima Terceros', 'Ingreso Materia Prima Neta'),
     ('Materia Prima Terceros', 'Unidades por kilo'),
     ('Materia Prima Terceros', '% Rechazo mmpp'),
     ('Materia Prima Terceros', '% MP Bruta Propia sobre Total MP Bruta'),
     ('Total Materia Prima', 'Ingreso Materia Prima Bruta'),
     ('Total Materia Prima', 'Ingreso Materia Prima Neta'),
     ('Total Materia Prima', '% Rechazo mmpp'),
     ],:].fillna("")
except:
    print('')
finally: 
    df = df.round(2).sort_index(ascending = [True, False])

NameError: name 'idx' is not defined

#### Mes Otras Industrias

In [48]:
Empresa = 'Sudmaris'
mes = 9

# Nuevas columnas
df = ingresos_mmpp[(ingresos_mmpp.Empresa != empresa) & (ingresos_mmpp.Mes == mes)].copy()
df['Indicadores'] = [dict_indicadores_2[df.Indicador.iloc[i]] for i in range(len(df.Indicador))]
df['Propietario'] = [dict_propiedad[df.Indicador.iloc[i]] for i in range(len(df.Indicador))]

# Añadir nuevas columnas para el total
new_df = pd.DataFrame(columns = df.columns)

for e in df.Empresa.unique():
    for m in df.Mes.unique():
        new_row = {
            'Indicador':'Ingreso Materia Prima Bruta', 
            'Valor': df[(df.Empresa == e) & (df.Mes == m) & (df.Tipo == 'MP Bruta')].Valor.sum(),
            'Unidad de medida':'Ton', 
            'Definición':'', 
            'Mes':m, 
            'Año':2021,
            'Empresa':e,
            'Tipo':'MP Bruta', 
            'Propiedad':'', 
            'Indicadores':'Ingreso Materia Prima Bruta', 
            'Propietario':'Total Materia Prima'
        }
        new_df = new_df.append(new_row, ignore_index = True)
        
        new_row = {
            'Indicador':'Ingreso Materia Prima Neta', 
            'Valor': df[(df.Empresa == e) & (df.Mes == m) & (df.Tipo == 'MP Neta')].Valor.sum(),
            'Unidad de medida':'Ton', 
            'Definición':'', 
            'Mes':m, 
            'Año':2021,
            'Empresa':e,
            'Tipo':'MP Bruta', 
            'Propiedad':'', 
            'Indicadores':'Ingreso Materia Prima Neta', 
            'Propietario':'Total Materia Prima'
        }
        new_df = new_df.append(new_row, ignore_index = True)
        
df = df.append(new_df, ignore_index=True)
df = df.groupby(['Propietario','Indicadores']).Valor.agg(['sum', np.mean], axis='columns')
df = df.round(2)
df.columns = ['Total', 'Promedio']

# Promedio ponderado unidades por kilo
df2 = ingresos_mmpp[(ingresos_mmpp.Empresa != empresa) & (ingresos_mmpp.Mes == mes)].copy()
df2['Indicadores'] = [dict_indicadores_2[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
df2['Propietario'] = [dict_propiedad[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
filtro = (df2.Indicadores == 'Ingreso Materia Prima Neta') | (df2.Indicadores=='Unidades por kilo')

df3 = df2[filtro].groupby(['Empresa','Mes','Propietario','Indicadores']).Valor.sum().unstack(level=-1).fillna(0)

df3_propia = df3.loc[idx[:,:,'Materia Prima Propia'],:]
uxk_propia = (df3_propia['Ingreso Materia Prima Neta']*df3_propia['Unidades por kilo']).sum()/df3_propia['Ingreso Materia Prima Neta'].sum()


df3_terceros = df3.loc[idx[:,:,'Materia Prima Terceros'],:]
uxk_terceros = (df3_terceros['Ingreso Materia Prima Neta']*df3_terceros['Unidades por kilo']).sum()/df3_terceros['Ingreso Materia Prima Neta'].sum()


#Cambios en Unidades por Kilo y %MP Bruta

for i, j in df.index:
    df.loc[(i,'% Rechazo mmpp'), 'Total'] = (df.loc[(i,'Ingreso Materia Prima Bruta'), 'Total']-df.loc[(i,'Ingreso Materia Prima Neta'), 'Total'])*100/df.loc[(i,'Ingreso Materia Prima Bruta'), 'Total']
    if i != 'Total Materia Prima':
        df.loc[(i,'% MP Bruta Propia sobre Total MP Bruta'),'Total'] = df.loc[(i,'Ingreso Materia Prima Bruta'), 'Total']/df.loc[('Total Materia Prima','Ingreso Materia Prima Bruta'),'Total']*100
    if (i == 'Materia Prima Propia') & (j=='Unidades por kilo'):
        # Promedio ponderado unidades por kilo
        df2 = ingresos_mmpp.copy()
        df2['Indicadores'] = [dict_indicadores_2[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        df2['Propietario'] = [dict_propiedad[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        filtro = (df2.Indicadores == 'Ingreso Materia Prima Neta') | (df2.Indicadores=='Unidades por kilo')
        df3 = df2[filtro].groupby(['Empresa','Mes','Propietario','Indicadores']).Valor.sum().unstack(level=-1).fillna(0)
        df3_propia = df3.loc[idx[:,:,'Materia Prima Propia'],:]
        uxk_propia = (df3_propia['Ingreso Materia Prima Neta']*df3_propia['Unidades por kilo']).sum()/df3_propia['Ingreso Materia Prima Neta'].sum()
        df.loc[(i,j),'Total'] = uxk_propia    
        
    if (i == 'Materia Prima Terceros') & (j=='Unidades por kilo'):
        # Promedio ponderado unidades por kilo
        df2 = ingresos_mmpp.copy()
        df2['Indicadores'] = [dict_indicadores_2[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        df2['Propietario'] = [dict_propiedad[df2.Indicador.iloc[i]] for i in range(len(df2.Indicador))]
        filtro = (df2.Indicadores == 'Ingreso Materia Prima Neta') | (df2.Indicadores=='Unidades por kilo')
        df3 = df2[filtro].groupby(['Empresa','Mes','Propietario','Indicadores']).Valor.sum().unstack(level=-1).fillna(0)
        df3_terceros = df3.loc[idx[:,:,'Materia Prima Terceros'],:]
        uxk_terceros = (df3_terceros['Ingreso Materia Prima Neta']*df3_terceros['Unidades por kilo']).sum()/df3_terceros['Ingreso Materia Prima Neta'].sum()
        df.loc[(i,j),'Total'] = uxk_terceros

df.round(2).sort_index(ascending = [True, False])


df = df.loc[[('Materia Prima Propia', 'Ingreso Materia Prima Bruta'),
 ('Materia Prima Propia', 'Ingreso Materia Prima Neta'),
 ('Materia Prima Propia', 'Unidades por kilo'),
 ('Materia Prima Propia', '% Rechazo mmpp'),
 ('Materia Prima Propia', '% MP Bruta Propia sobre Total MP Bruta'),
 ('Materia Prima Terceros', 'Ingreso Materia Prima Bruta'),
 ('Materia Prima Terceros', 'Ingreso Materia Prima Neta'),
 ('Materia Prima Terceros', 'Unidades por kilo'),
 ('Materia Prima Terceros', '% Rechazo mmpp'),
 ('Materia Prima Terceros', '% MP Bruta Propia sobre Total MP Bruta'),
 ('Total Materia Prima', 'Ingreso Materia Prima Bruta'),
 ('Total Materia Prima', 'Ingreso Materia Prima Neta'),
 ('Total Materia Prima', '% Rechazo mmpp'),
 ],:].fillna("")

df.round(2)

NameError: name 'idx' is not defined

# Gráficos Total Materia Prima Bruta vs Neta

In [49]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches

datos = ingresos_mmpp[(ingresos_mmpp.Tipo== 'MP Bruta') | (ingresos_mmpp.Tipo== 'MP Neta')].copy()[['Empresa','Tipo','Mes','Valor']]
datos.head()

,Empresa,Tipo,Mes,Valor
0,SA Chonchi,MP Bruta,9,269.442
1,SA Chonchi,MP Neta,9,223.038
3,SA Chonchi,MP Bruta,9,362.001
4,SA Chonchi,MP Neta,9,302.073
6,SA Chonchi,MP Bruta,6,3099.771


In [50]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

datos_w=pd.pivot_table(datos, index='Mes', columns='Tipo', values='Valor', aggfunc=np.sum)

fig_industria = px.bar(datos_w, x=datos_w.index,
            y=['MP Bruta','MP Neta'], title="Total Materia Prima Bruta vs Neta",
            labels={'Mes':'Mes','value':'Ton','variable':'Tipo'},
            color_discrete_map={"MP Bruta": "#00609C", "MP Neta": "#F4D19F"},
            template="simple_white",
            width=700, 
            height=500,
            )

fig_industria.update_layout(yaxis_range=[0,60000])

fig_industria.show()

In [51]:
import plotly.express as px

empresa = 'Sudmaris'

datos_w=pd.pivot_table(datos[datos.Empresa == empresa], index='Mes', columns='Tipo', values='Valor', aggfunc=np.sum)

fig_empresa = px.bar(datos_w, x=datos_w.index,
            y=['MP Bruta','MP Neta'], title="Total Materia Prima Bruta vs Neta "+empresa,
            labels={'Mes':'Mes','value':'Ton','variable':'Tipo'},
            color_discrete_map={"MP Bruta": "#C45130", "MP Neta": "#FFAA61"},
            template="simple_white",
            width=700, 
            height=500,
            )

fig_empresa.show()

In [52]:
import plotly.subplots as sp
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

figure1_traces = []
figure2_traces = []
for trace in range(len(fig_industria["data"])):
    figure1_traces.append(fig_industria["data"][trace])
for trace in range(len(fig_empresa["data"])):
    figure2_traces.append(fig_empresa["data"][trace])

#Create a 1x2 subplot
this_figure = sp.make_subplots(rows=1, cols=2) 

# Get the Express fig broken down as traces and add the traces to the proper plot within in the subplot
for traces in figure1_traces:
    this_figure.append_trace(traces, row=1, col=1)
for traces in figure2_traces:
    this_figure.append_trace(traces, row=1, col=2)

#the subplot as shown in the above image
final_graph = dcc.Graph(figure=this_figure)

# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    final_graph
    ])
# Define callback to update graph


app.run_server(mode='inline')

ModuleNotFoundError: No module named 'jupyter_dash'

# Indicador Materia Prima Bruta por origen

In [53]:

datos = ingresos_mmpp[(ingresos_mmpp.Tipo== 'MP Bruta')].copy()[['Empresa','Propiedad','Mes','Valor']]
datos.head()

,Empresa,Propiedad,Mes,Valor
0,SA Chonchi,MPB Propia,9,269.442
3,SA Chonchi,MPB Terceros,9,362.001
6,SA Chonchi,MPB Propia,6,3099.771
9,SA Chonchi,MPB Terceros,6,1434.065
12,SA Chonchi,MPB Propia,5,5896.360


In [54]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

datos_w=pd.pivot_table(datos, index='Mes', columns='Propiedad', values='Valor', aggfunc=np.sum)

fig_industria = px.bar(datos_w, x=datos_w.index,
            y=['MPB Propia','MPB Terceros'], title="Indicador Materia Prima Bruta por origen",
            labels={'Mes':'Mes','value':'Ton','variable':'Origen'},
            color_discrete_map={"MPB Propia": "#00609C", "MPB Terceros": "#F4D19F"},
            template="simple_white",
            width=700, 
            height=500,
            )

fig_industria.show()

In [55]:
empresa = 'Manuelita'
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

datos_w=pd.pivot_table(datos[datos.Empresa == empresa], index='Mes', columns='Propiedad', values='Valor', aggfunc=np.sum)

fig_industria = px.bar(datos_w, x=datos_w.index,
            y=['MPB Propia','MPB Terceros'], title="Indicador Materia Prima Bruta por origen "+empresa,
            labels={'Mes':'Mes','value':'Ton','variable':'Origen'},
            color_discrete_map={"MPB Propia": "#C45130", "MPB Terceros": "#FFAA61"},
            template="simple_white",
            width=700, 
            height=500)

fig_industria.show()

# Unidades por Kilo

In [56]:
datos = ingresos_mmpp.copy()
datos['Indicadores'] = [dict_indicadores_2[datos.Indicador.iloc[i]] for i in range(len(datos.Indicador))]
datos['Propietario'] = [dict_propiedad[datos.Indicador.iloc[i]] for i in range(len(datos.Indicador))]
datos.drop_duplicates(inplace = True, subset=['Indicador','Valor','Mes','Año','Empresa'])

In [57]:
datos.drop_duplicates(inplace = True, subset=['Indicador','Valor','Mes','Año','Empresa'])

In [58]:
datos.loc[datos.Empresa == 'Sudmaris']

,Indicador,Valor,Unidad de medida,Definición,Mes,Año,Empresa,Tipo,Propiedad,Indicadores,Propietario
238,Ingreso Materia Prima Bruta Propia,1001.615500,Ton,Total Kilos mmpp de centros de la empresa ingr...,1,2021,Sudmaris,MP Bruta,MPB Propia,Ingreso Materia Prima Bruta,Materia Prima Propia
239,Ingreso Materia Prima Neta o Pagada Propia,742.000986,Ton,Total Kilos mmpp de centros de la empresa ingr...,1,2021,Sudmaris,MP Neta,MPN Propia,Ingreso Materia Prima Neta,Materia Prima Propia
240,Unidades por kilo mmpp propia*,65.000000,uni/kg,Promedio Ponderado del Número de Mejillones po...,1,2021,Sudmaris,No aplica,No aplica,Unidades por kilo,Materia Prima Propia
241,Ingreso Materia Prima Bruta Terceros,2371.829000,Ton,Total Kilos mmpp de centros de terceros ingres...,1,2021,Sudmaris,MP Bruta,MPB Terceros,Ingreso Materia Prima Bruta,Materia Prima Terceros
242,Ingreso Materia Prima Neta o Pagada Terceros,1996.818919,Ton,Total Kilos mmpp de centros de terceros ingre...,1,2021,Sudmaris,MP Neta,MPN Terceros,Ingreso Materia Prima Neta,Materia Prima Terceros
243,Unidades por kilo mmpp terceros*,67.000000,uni/kg,Promedio Ponderado del Número de Mejillones po...,1,2021,Sudmaris,No aplica,No aplica,Unidades por kilo,Materia Prima Terceros
245,Ingreso Materia Prima Bruta Propia,1385.808000,Ton,Total Kilos mmpp de centros de la empresa ingr...,2,2021,Sudmaris,MP Bruta,MPB Propia,Ingreso Materia Prima Bruta,Materia Prima Propia
246,Ingreso Materia Prima Neta o Pagada Propia,935.469000,Ton,Total Kilos mmpp de centros de la empresa ingr...,2,2021,Sudmaris,MP Neta,MPN Propia,Ingreso Materia Prima Neta,Materia Prima Propia
247,Unidades por kilo mmpp propia*,61.000000,uni/kg,Promedio Ponderado del Número de Mejillones po...,2,2021,Sudmaris,No aplica,No aplica,Unidades por kilo,Materia Prima Propia
248,Ingreso Materia Prima Bruta Terceros,2563.012000,Ton,Total Kilos mmpp de centros de terceros ingres...,2,2021,Sudmaris,MP Bruta,MPB Terceros,Ingreso Materia Prima Bruta,Materia Prima Terceros


In [59]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

empresa = 'Sudmaris'

datos_w = datos[(datos.Empresa == empresa) & (datos.Indicadores == 'Unidades por kilo')].groupby(['Mes','Propietario']).Valor.sum().unstack()
datos_w.drop_duplicates(inplace = True)

fig_industria = px.bar(datos_w, x=datos_w.index,
            y=['Materia Prima Propia','Materia Prima Terceros'], title="Indicador Materia Prima Bruta por origen "+empresa,
            labels={'Mes':'Mes','value':'Unidades por kilo','variable':'Origen'},
            color_discrete_map={"Materia Prima Propia": "#C45130", "Materia Prima Terceros": "#FFAA61"},
            template="simple_white",
            width=700, 
            height=500)

fig_industria.show()

In [63]:
datos2 = datos[datos.Indicadores != 'Ingreso Materia Prima Bruta']
datos2 = pd.pivot_table(datos2, index=['Mes','Empresa'], columns=['Propietario','Indicadores'], values='Valor').fillna(0)
idx = pd.IndexSlice

In [64]:
datos2.loc[idx[1,:],:]['Materia Prima Propia']['Ingreso Materia Prima Neta'] * datos2.loc[idx[1,:],:]['Materia Prima Propia']['Unidades por kilo']

Mes  Empresa   
1    Blue Shell    112731.355000
     Camanchaca    134898.093000
     Landes         24603.179800
     Manuelita     132838.322469
     SA Chonchi    185443.664000
     SA Quemchi    135977.238000
     Sudmaris       48230.064090
dtype: float64

In [65]:
df_unidades = pd.DataFrame(columns = ['Mes','Unidades por kilo Propia', 'Unidades por kilo Terceros'])

for i,j in datos2.index:
    df_aux = datos2.loc[idx[i,:],:]['Materia Prima Propia']
    df_auxa = datos2.loc[idx[i,:],:]['Materia Prima Terceros']
    new_row = {
        'Mes':i,
        'Unidades por kilo Propia': (df_aux['Ingreso Materia Prima Neta']*df_aux['Unidades por kilo']).sum()/df_aux['Ingreso Materia Prima Neta'].sum(),
        'Unidades por kilo Terceros': (df_auxa['Ingreso Materia Prima Neta']*df_auxa['Unidades por kilo']).sum()/df_auxa['Ingreso Materia Prima Neta'].sum()
    }
    df_unidades = df_unidades.append(new_row, ignore_index = True)

In [66]:
df_unidades.drop_duplicates(inplace = True)
fig_industria = px.bar(df_unidades, x='Mes',
            y=['Unidades por kilo Propia','Unidades por kilo Terceros'], title="Unidades por kilo ",
            labels={'Mes':'Mes','value':'Unidades por kilo','variable':'Origen'},
            color_discrete_map={"Unidades por kilo Propia": "#00609C", "Unidades por kilo Terceros": "#F4D19F"},
            template="simple_white",
            width=700, 
            height=500)
fig_industria.show()

# Rechazo por origen

In [67]:
datos = ingresos_mmpp.copy()
datos['Indicadores'] = [dict_indicadores_2[datos.Indicador.iloc[i]] for i in range(len(datos.Indicador))]
datos['Propietario'] = [dict_propiedad[datos.Indicador.iloc[i]] for i in range(len(datos.Indicador))]
datos.drop_duplicates(inplace = True, subset=['Indicador','Valor','Mes','Año','Empresa'])

In [68]:
datos = pd.pivot_table(datos[datos.Indicadores != 'Unidades por kilo'], index=['Mes'], columns=['Propietario','Indicadores'], values='Valor')
for i in datos.index:
    datos.loc[i,('Materia Prima Propia','% Rechazo mmpp')] = (datos.loc[i,('Materia Prima Propia','Ingreso Materia Prima Bruta')] - datos.loc[i,('Materia Prima Propia','Ingreso Materia Prima Neta')]) /datos.loc[i,('Materia Prima Propia','Ingreso Materia Prima Bruta')]*100
    datos.loc[i,('Materia Prima Terceros','% Rechazo mmpp')] = (datos.loc[i,('Materia Prima Terceros','Ingreso Materia Prima Bruta')] - datos.loc[i,('Materia Prima Terceros','Ingreso Materia Prima Neta')]) /datos.loc[i,('Materia Prima Propia','Ingreso Materia Prima Bruta')]*100

datos = datos.loc[idx[:],idx[:,'% Rechazo mmpp']]
datos.columns = ['% Rechazo mmpp Propia','% Rechazo mmpp Terceros']

In [69]:
import plotly.express as px
fig = px.line(datos, x=datos.index, y=['% Rechazo mmpp Propia','% Rechazo mmpp Terceros'], markers=True,
                title="Porcentaje de rechazo por origen",
                labels={'Mes':'Mes','value':'% Rechazo MMPP','variable':'Origen'},
                color_discrete_map={'% Rechazo mmpp Propia': "#00609C", '% Rechazo mmpp Terceros': "#F4D19F"},
                template="simple_white",
                width=1000, 
                height=500,)
fig.show()

In [70]:
datos = ingresos_mmpp[ingresos_mmpp.Empresa == empresa].copy()
datos['Indicadores'] = [dict_indicadores_2[datos.Indicador.iloc[i]] for i in range(len(datos.Indicador))]
datos['Propietario'] = [dict_propiedad[datos.Indicador.iloc[i]] for i in range(len(datos.Indicador))]
datos.drop_duplicates(inplace = True, subset=['Indicador','Valor','Mes','Año','Empresa'])

datos = pd.pivot_table(datos[datos.Indicadores != 'Unidades por kilo'], index=['Mes'], columns=['Propietario','Indicadores'], values='Valor')
for i in datos.index:
    datos.loc[i,('Materia Prima Propia','% Rechazo mmpp')] = (datos.loc[i,('Materia Prima Propia','Ingreso Materia Prima Bruta')] - datos.loc[i,('Materia Prima Propia','Ingreso Materia Prima Neta')]) /datos.loc[i,('Materia Prima Propia','Ingreso Materia Prima Bruta')]*100
    datos.loc[i,('Materia Prima Terceros','% Rechazo mmpp')] = (datos.loc[i,('Materia Prima Terceros','Ingreso Materia Prima Bruta')] - datos.loc[i,('Materia Prima Terceros','Ingreso Materia Prima Neta')]) /datos.loc[i,('Materia Prima Propia','Ingreso Materia Prima Bruta')]*100

datos = datos.loc[idx[:],idx[:,'% Rechazo mmpp']]
datos.columns = ['% Rechazo mmpp Propia','% Rechazo mmpp Terceros']

import plotly.express as px
fig = px.line(datos, x=datos.index, y=['% Rechazo mmpp Propia','% Rechazo mmpp Terceros'], markers=True,
                title="Porcentaje de rechazo por origen "+empresa,
                labels={'Mes':'Mes','value':'% Rechazo MMPP','variable':'Origen'},
                color_discrete_map={'% Rechazo mmpp Propia': "#00609C", '% Rechazo mmpp Terceros': "#F4D19F"},
                template="simple_white",
                width=1000, 
                height=500,)
fig.show()

C:\Users\erossel\AppData\Local\Temp/ipykernel_15932/2249708562.py:8: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\erossel\AppData\Local\Temp/ipykernel_15932/2249708562.py:9: RuntimeWarning:

divide by zero encountered in double_scalars



# Productos

### Industria

In [900]:
producto = pd.read_csv('../Procesos_Producto_2021.csv')
producto.head()

filtro = (producto.Producto == 'Carne') | (producto.Producto == 'Entero') | (producto.Producto == 'Media Concha')
datos = producto.loc[filtro].pivot_table(index='Producto',columns='Mes', values='Valor')
total = datos.sum(axis=1).round(0)
average = datos.mean(axis=1).round(0)
datos[13] = total
datos[14] = average
datos

filtro = (producto.Producto == 'Carne') | (producto.Producto == 'Entero') | (producto.Producto == 'Media Concha')
datos2 = producto.loc[filtro].pivot_table(index='Producto',columns='Mes', values='Valor')
total2 = datos2.sum(axis=1).round(0)
average2 = datos2.mean(axis=1).round(0)
datos2[13] = total2
datos2[14] = average2
datos2 = datos2.transform(lambda x: (x / x.sum()).round(2))*100
datos2

largo = len(datos.columns)
new_index = []
for i in range(1, largo+1):
    if i == largo - 1:
        new_index.append(('Total','Ton'))
        new_index.append(('Total','%'))
    elif i == largo:
        new_index.append(('Promedio','Ton'))
        new_index.append(('Promedio','%'))
    else: 
        new_index.append((dict_meses[i],'Ton'))
        new_index.append((dict_meses[i],'%'))

pv3 = pd.concat([datos, datos2], axis=1).sort_index(level=0,axis=1)
pv3.columns = pd.MultiIndex.from_tuples(new_index)
pv3.round(1)

largo = len(pv3.columns)
for i in range(1,largo+1,2):
    if pv3.iloc[:,i].sum() != 100:
        pv3.iloc[:,[i-1, i]] = pv3.iloc[:,[i,i-1]].copy() 
        
pv3 = pv3.round(1)
pv3


Enero         Febrero           Marzo           Abril        \
                   Ton     %       Ton     %       Ton     %       Ton     %   
Producto                                                                       
Carne         131195.1  59.0  132293.9  60.0  160544.1  58.0  149826.1  56.0   
Entero         85025.3  38.0   83631.9  38.0  112798.1  41.0  112999.6  42.0   
Media Concha    6744.2   3.0    3419.4   2.0    3691.5   1.0    5699.2   2.0   

              Mayo            ... Octubre     Noviembre     Diciembre      \
               Ton         %  ...     Ton   %       Ton   %       Ton   %   
Producto                      ...                                           
Carne         60.0  143917.7  ...     0.0 NaN       0.0 NaN       0.0 NaN   
Entero        39.0   92557.4  ...     0.0 NaN       0.0 NaN       0.0 NaN   
Media Concha   2.0    3850.3  ...     0.0 NaN       0.0 NaN       0.0 NaN   

                 Total       Promedio        
                   Ton     %      Ton     %  
Producto                                     
Carne         998051.0  53.0  83171.0  53.0  
Entero        852907.0  45.0  71076.0  45.0  
Media Concha   34825.0   2.0   2902.0   2.0  

[3 rows x 28 columns]

### Empresa

In [870]:
empresa = 'Sudmaris'
filtro = (producto.Empresa == empresa) & ((producto.Producto == 'Carne') | (producto.Producto == 'Entero') | (producto.Producto == 'Media Concha'))
datos = producto.loc[filtro].pivot_table(index='Producto',columns='Mes', values='Valor')
total = datos.sum(axis=1).round(0)
average = datos.mean(axis=1).round(0)
datos[13] = total
datos[14] = average
datos

filtro = (producto.Producto == 'Carne') | (producto.Producto == 'Entero') | (producto.Producto == 'Media Concha')
datos2 = producto.loc[filtro].pivot_table(index='Producto',columns='Mes', values='Valor')
total2 = datos2.sum(axis=1).round(0)
average2 = datos2.mean(axis=1).round(0)
datos2[13] = total2
datos2[14] = average2
datos2 = datos2.transform(lambda x: (x / x.sum()).round(2))*100
datos2

largo = len(datos.columns)
new_index = []
for i in range(1, largo+1):
    if i == largo - 1:
        new_index.append(('Total','Ton'))
        new_index.append(('Total','%'))
    elif i == largo:
        new_index.append(('Promedio','Ton'))
        new_index.append(('Promedio','%'))
    else: 
        new_index.append((dict_meses[i],'Ton'))
        new_index.append((dict_meses[i],'%'))

pv3 = pd.concat([datos, datos2], axis=1).sort_index(level=0,axis=1)
pv3.columns = pd.MultiIndex.from_tuples(new_index)
pv3.round(1)

largo = len(pv3.columns)
for i in range(1,largo+1,2):
    if pv3.iloc[:,i].sum() != 100:
        pv3.iloc[:,[i-1, i]] = pv3.iloc[:,[i,i-1]].copy() 
        
pv3 = pv3.round(1)
pv3


Enero        Febrero           Marzo          Abril        \
                   Ton     %      Ton     %       Ton     %      Ton     %   
Producto                                                                     
Carne              0.0  59.0      0.0  60.0       0.0  58.0      0.0  56.0   
Entero        106228.0  38.0  75970.5  38.0  120903.1  41.0  93752.1  42.0   
Media Concha    3069.0   3.0   4807.0   2.0    2299.0   1.0   1503.3   2.0   

              Mayo           ... Octubre     Noviembre     Diciembre      \
               Ton        %  ...     Ton   %       Ton   %       Ton   %   
Producto                     ...                                           
Carne         60.0      0.0  ...     0.0 NaN       0.0 NaN       0.0 NaN   
Entero        39.0  88610.5  ...     0.0 NaN       0.0 NaN       0.0 NaN   
Media Concha   2.0   1188.0  ...     0.0 NaN       0.0 NaN       0.0 NaN   

                 Total       Promedio        
                   Ton     %      Ton     %  
Producto                                     
Carne              0.0  53.0      0.0  53.0  
Entero        921661.0  45.0  76805.0  45.0  
Media Concha   14568.0   2.0   1214.0   2.0  

[3 rows x 28 columns]

### Otras Industrias

In [869]:
empresa = 'Sudmaris'
filtro = (producto.Empresa != empresa) & ((producto.Producto == 'Carne') | (producto.Producto == 'Entero') | (producto.Producto == 'Media Concha'))
datos = producto.loc[filtro].pivot_table(index='Producto',columns='Mes', values='Valor')
total = datos.sum(axis=1).round(0)
average = datos.mean(axis=1).round(0)
datos[13] = total
datos[14] = average
datos

filtro = (producto.Producto == 'Carne') | (producto.Producto == 'Entero') | (producto.Producto == 'Media Concha')
datos2 = producto.loc[filtro].pivot_table(index='Producto',columns='Mes', values='Valor')
total2 = datos2.sum(axis=1).round(0)
average2 = datos2.mean(axis=1).round(0)
datos2[13] = total2
datos2[14] = average2
datos2 = datos2.transform(lambda x: (x / x.sum()).round(2))*100
datos2

largo = len(datos.columns)
new_index = []
for i in range(1, largo+1):
    if i == largo - 1:
        new_index.append(('Total','Ton'))
        new_index.append(('Total','%'))
    elif i == largo:
        new_index.append(('Promedio','Ton'))
        new_index.append(('Promedio','%'))
    else: 
        new_index.append((dict_meses[i],'Ton'))
        new_index.append((dict_meses[i],'%'))

pv3 = pd.concat([datos, datos2], axis=1).sort_index(level=0,axis=1)
pv3.dropna(axis=1, inplace=True, thresh = 1)
pv3.columns = pd.MultiIndex.from_tuples(new_index)
pv3.round(1)

largo = len(pv3.columns)
for i in range(1,largo+1,2):
    if pv3.iloc[:,i].sum() != 100:
        pv3.iloc[:,[i-1, i]] = pv3.iloc[:,[i,i-1]].copy() 
        
pv3 = pv3.round(1)
pv3



Enero         Febrero           Marzo           Abril        \
                   Ton     %       Ton     %       Ton     %       Ton     %   
Producto                                                                       
Carne         141690.8  59.0  142877.4  60.0  176598.5  58.0  165597.3  56.0   
Entero         80481.8  38.0   85273.7  38.0  110587.6  41.0  119415.4  42.0   
Media Concha    7269.2   3.0    3141.9   2.0    4012.8   1.0    7272.6   2.0   

              Mayo            ...     Julio         Agosto       Septiembre  \
               Ton         %  ...       Ton     %      Ton     %        Ton   
Producto                      ...                                             
Carne         60.0  158309.4  ...  123203.4  49.0  36672.5  30.0    19427.9   
Entero        39.0   93873.1  ...  107696.4  50.0  68919.1  67.0    82510.4   
Media Concha   2.0    4649.0  ...    2716.3   1.0   3633.8   3.0     3849.0   

                        Total        Promedio        
                 %        Ton     %       Ton     %  
Producto                                             
Carne         16.0  1094508.0  53.0  121612.0  53.0  
Entero        81.0   838252.0  45.0   93139.0  45.0  
Media Concha   3.0    41134.0   2.0    4570.0   2.0  

[3 rows x 22 columns]

# Detalle Producto

In [915]:
producto = pd.read_csv('../Procesos_Producto_2021.csv')
producto.drop_duplicates(inplace=True)
filtro = (producto.Producto == 'Carne') | (producto.Producto == 'Entero') | (producto.Producto == 'Media Concha')
producto.drop_duplicates(inplace=True)
datos = producto.loc[filtro].pivot_table(index=['Producto','Calibre'], values='Valor')

for p, c in datos.index:
    datos.loc[(p, c),'%'] = datos.loc[(p, c),'Valor']*100/datos.loc[idx[p,:],'Valor'].sum()
    
#Reindex
new_index = [(       'Carne',   '50-100 UNI/KG'),
            (       'Carne',  '100-200 UNI/KG'),
            (       'Carne',  '200-300 UNI/KG'),
            (       'Carne',  '300-500 UNI/KG'),
            (       'Carne', '500-1000 UNI/KG'),
            (       'Carne',      'INDUSTRIAL'),
            (      'Entero',       'CON SALSA'),
            (      'Entero',         'NATURAL'),
            ('Media Concha',    '40-60 UNI/KG'),
            ('Media Concha',    '60-80 UNI/KG'),
            ('Media Concha',   '80-100 UNI/KG'),
            ('Media Concha',   '100 UP UNI/KG')]

datos = datos.loc[new_index]

datos = pd.concat([
    d.append(d.sum().rename((k, 'Total')))
    for k, d in datos.groupby(level=0)
]).append(datos.sum().rename(('General', 'Total'))).round()

datos

Valor      %
Producto     Calibre                         
Carne        50-100 UNI/KG      1444.0    0.0
             100-200 UNI/KG   214333.0   27.0
             200-300 UNI/KG   342066.0   43.0
             300-500 UNI/KG   187196.0   24.0
             500-1000 UNI/KG   10315.0    1.0
             INDUSTRIAL        37586.0    5.0
             Total            792940.0  100.0
Entero       CON SALSA         35806.0   20.0
             NATURAL          143297.0   80.0
             Total            179103.0  100.0
Media Concha 40-60 UNI/KG       3781.0   20.0
             60-80 UNI/KG      12604.0   67.0
             80-100 UNI/KG      2260.0   12.0
             100 UP UNI/KG       171.0    1.0
             Total             18815.0  100.0
General      Total            990858.0  300.0

### Empresa

In [917]:
producto = pd.read_csv('../Procesos_Producto_2021.csv')
producto.drop_duplicates(inplace=True)
empresa = 'Sudmaris'

filtro = (producto.Empresa == empresa) & ((producto.Producto == 'Carne') | (producto.Producto == 'Entero') | (producto.Producto == 'Media Concha'))
producto.drop_duplicates(inplace=True)
datos = producto.loc[filtro].pivot_table(index=['Producto','Calibre'], values='Valor')

for p, c in datos.index:
    datos.loc[(p, c),'%'] = datos.loc[(p, c),'Valor']*100/datos.loc[idx[p,:],'Valor'].sum()
    
#Reindex
try: 
    new_index = [(       'Carne',   '50-100 UNI/KG'),
            (       'Carne',  '100-200 UNI/KG'),
            (       'Carne',  '200-300 UNI/KG'),
            (       'Carne',  '300-500 UNI/KG'),
            (       'Carne', '500-1000 UNI/KG'),
            (       'Carne',      'INDUSTRIAL'),
            (      'Entero',       'CON SALSA'),
            (      'Entero',         'NATURAL'),
            ('Media Concha',    '40-60 UNI/KG'),
            ('Media Concha',    '60-80 UNI/KG'),
            ('Media Concha',   '80-100 UNI/KG'),
            ('Media Concha',   '100 UP UNI/KG')]

    datos = datos.loc[new_index]

    datos = pd.concat([
        d.append(d.sum().rename((k, 'Total')))
        for k, d in datos.groupby(level=0)
    ]).append(datos.sum().rename(('General', 'Total'))).round()
except:
    datos = pd.concat([
        d.append(d.sum().rename((k, 'Total')))
        for k, d in datos.groupby(level=0)
    ]).append(datos.sum().rename(('General', 'Total'))).round()
    
datos

/var/folders/f8/dscw5x7s14n8dmf67ydf66zr0000gn/T/ipykernel_1625/3174729568.py:10: RuntimeWarning:

invalid value encountered in double_scalars



Valor      %
Producto     Calibre                      
Carne        50-100 UNI/KG      0.0    NaN
             Total              0.0    0.0
Entero       NATURAL        95344.0  100.0
             Total          95344.0  100.0
Media Concha 40-60 UNI/KG    2032.0  100.0
             Total           2032.0  100.0
General      Total          97376.0  200.0

### Otras industrias

In [918]:
producto = pd.read_csv('../Procesos_Producto_2021.csv')
producto.drop_duplicates(inplace=True)
empresa = 'Sudmaris'

filtro = (producto.Empresa != empresa) & ((producto.Producto == 'Carne') | (producto.Producto == 'Entero') | (producto.Producto == 'Media Concha'))
producto.drop_duplicates(inplace=True)
datos = producto.loc[filtro].pivot_table(index=['Producto','Calibre'], values='Valor')

for p, c in datos.index:
    datos.loc[(p, c),'%'] = datos.loc[(p, c),'Valor']*100/datos.loc[idx[p,:],'Valor'].sum()
    
#Reindex
try: 
    new_index = [(       'Carne',   '50-100 UNI/KG'),
            (       'Carne',  '100-200 UNI/KG'),
            (       'Carne',  '200-300 UNI/KG'),
            (       'Carne',  '300-500 UNI/KG'),
            (       'Carne', '500-1000 UNI/KG'),
            (       'Carne',      'INDUSTRIAL'),
            (      'Entero',       'CON SALSA'),
            (      'Entero',         'NATURAL'),
            ('Media Concha',    '40-60 UNI/KG'),
            ('Media Concha',    '60-80 UNI/KG'),
            ('Media Concha',   '80-100 UNI/KG'),
            ('Media Concha',   '100 UP UNI/KG')]

    datos = datos.loc[new_index]

    datos = pd.concat([
        d.append(d.sum().rename((k, 'Total')))
        for k, d in datos.groupby(level=0)
    ]).append(datos.sum().rename(('General', 'Total'))).round()
except:
    datos = pd.concat([
        d.append(d.sum().rename((k, 'Total')))
        for k, d in datos.groupby(level=0)
    ]).append(datos.sum().rename(('General', 'Total'))).round()
    
datos

Valor      %
Producto     Calibre                          
Carne        50-100 UNI/KG       2003.0    0.0
             100-200 UNI/KG    214333.0   27.0
             200-300 UNI/KG    342066.0   43.0
             300-500 UNI/KG    187196.0   24.0
             500-1000 UNI/KG    10315.0    1.0
             INDUSTRIAL         37586.0    5.0
             Total             793499.0  100.0
Entero       CON SALSA          35806.0   17.0
             NATURAL           172885.0   83.0
             Total             208691.0  100.0
Media Concha 40-60 UNI/KG        5076.0   25.0
             60-80 UNI/KG       12604.0   63.0
             80-100 UNI/KG       2260.0   11.0
             100 UP UNI/KG        171.0    1.0
             Total              20111.0  100.0
General      Total            1022301.0  300.0

### Solo último mes

In [920]:
producto = pd.read_csv('../Procesos_Producto_2021.csv')
producto.drop_duplicates(inplace=True)
mes = 7
filtro = (producto.Mes == mes) & ((producto.Producto == 'Carne') | (producto.Producto == 'Entero') | (producto.Producto == 'Media Concha'))
producto.drop_duplicates(inplace=True)
datos = producto.loc[filtro].pivot_table(index=['Producto','Calibre'], values='Valor')

for p, c in datos.index:
    datos.loc[(p, c),'%'] = datos.loc[(p, c),'Valor']*100/datos.loc[idx[p,:],'Valor'].sum()
    
#Reindex
new_index = [(       'Carne',   '50-100 UNI/KG'),
            (       'Carne',  '100-200 UNI/KG'),
            (       'Carne',  '200-300 UNI/KG'),
            (       'Carne',  '300-500 UNI/KG'),
            (       'Carne', '500-1000 UNI/KG'),
            (       'Carne',      'INDUSTRIAL'),
            (      'Entero',       'CON SALSA'),
            (      'Entero',         'NATURAL'),
            ('Media Concha',    '40-60 UNI/KG'),
            ('Media Concha',    '60-80 UNI/KG'),
            ('Media Concha',   '80-100 UNI/KG'),
            ('Media Concha',   '100 UP UNI/KG')]

datos = datos.loc[new_index]

datos = pd.concat([
    d.append(d.sum().rename((k, 'Total')))
    for k, d in datos.groupby(level=0)
]).append(datos.sum().rename(('General', 'Total'))).round()

datos

Valor      %
Producto     Calibre                          
Carne        50-100 UNI/KG       1700.0    0.0
             100-200 UNI/KG    106160.0   14.0
             200-300 UNI/KG    253276.0   32.0
             300-500 UNI/KG    372683.0   48.0
             500-1000 UNI/KG    16597.0    2.0
             INDUSTRIAL         29307.0    4.0
             Total             779722.0  100.0
Entero       CON SALSA          11373.0    5.0
             NATURAL           204226.0   95.0
             Total             215600.0  100.0
Media Concha 40-60 UNI/KG         782.0    7.0
             60-80 UNI/KG        8856.0   76.0
             80-100 UNI/KG       2022.0   17.0
             100 UP UNI/KG         18.0    0.0
             Total              11678.0  100.0
General      Total            1006999.0  300.0

In [921]:
producto = pd.read_csv('../Procesos_Producto_2021.csv')
producto.drop_duplicates(inplace=True)
empresa = 'Sudmaris'
mes = 7

filtro = (producto.Mes != mes) & (producto.Empresa != empresa) & ((producto.Producto == 'Carne') | (producto.Producto == 'Entero') | (producto.Producto == 'Media Concha'))
producto.drop_duplicates(inplace=True)
datos = producto.loc[filtro].pivot_table(index=['Producto','Calibre'], values='Valor')

for p, c in datos.index:
    datos.loc[(p, c),'%'] = datos.loc[(p, c),'Valor']*100/datos.loc[idx[p,:],'Valor'].sum()
    
#Reindex
try: 
    new_index = [(       'Carne',   '50-100 UNI/KG'),
            (       'Carne',  '100-200 UNI/KG'),
            (       'Carne',  '200-300 UNI/KG'),
            (       'Carne',  '300-500 UNI/KG'),
            (       'Carne', '500-1000 UNI/KG'),
            (       'Carne',      'INDUSTRIAL'),
            (      'Entero',       'CON SALSA'),
            (      'Entero',         'NATURAL'),
            ('Media Concha',    '40-60 UNI/KG'),
            ('Media Concha',    '60-80 UNI/KG'),
            ('Media Concha',   '80-100 UNI/KG'),
            ('Media Concha',   '100 UP UNI/KG')]

    datos = datos.loc[new_index]

    datos = pd.concat([
        d.append(d.sum().rename((k, 'Total')))
        for k, d in datos.groupby(level=0)
    ]).append(datos.sum().rename(('General', 'Total'))).round()
except:
    datos = pd.concat([
        d.append(d.sum().rename((k, 'Total')))
        for k, d in datos.groupby(level=0)
    ]).append(datos.sum().rename(('General', 'Total'))).round()
    
datos

Valor      %
Producto     Calibre                          
Carne        50-100 UNI/KG       1974.0    0.0
             100-200 UNI/KG    226815.0   29.0
             200-300 UNI/KG    352311.0   44.0
             300-500 UNI/KG    165793.0   21.0
             500-1000 UNI/KG     9590.0    1.0
             INDUSTRIAL         38542.0    5.0
             Total             795025.0  100.0
Entero       CON SALSA          39296.0   19.0
             NATURAL           164716.0   81.0
             Total             204012.0  100.0
Media Concha 40-60 UNI/KG        5376.0   26.0
             60-80 UNI/KG       13020.0   62.0
             80-100 UNI/KG       2288.0   11.0
             100 UP UNI/KG        202.0    1.0
             Total              20886.0  100.0
General      Total            1019923.0  300.0

# Dotación

In [929]:
dotacion = pd.read_csv('../Procesos_Dotacion_2021.csv')
dotacion.head()

dict_cargos =  {
    'Operarios directos (producción)': 'Operarios Directos',
    'Jefes de Turno': 'Supervisores Directos',
    'Supervisores Producción': 'Supervisores Directos',
    'Operarios Indirectos (Frigorífico, Aseo, etc)': 'Personal Indirecto',
    'Supervisores Calidad': 'Personal Indirecto',
    'Monitores Calidad': 'Personal Indirecto',
    'Operadores Mantención y Administración Directo Planta (Contador': 'Personal Indirecto',
    'Tesorero': 'Personal Indirecto',
    'Asistente Contabilidad': 'Personal Indirecto',
    'Jefe RRHH': 'Personal Indirecto',
    'Jefe de Adquisición': 'Personal Indirecto',
    'Prevención de Riesgo': 'Personal Indirecto',
    'Control de Gestión': 'Personal Indirecto',
    'Otro cargo que su trabajo esté asociado directamente con Planta': 'Personal Indirecto',
}

dotacion['Grupo'] = dotacion.Cargo.apply(lambda c: dict_cargos[c])
dotacion['Tipo'] = dotacion.Grupo.apply(lambda g: 'Personal Directo' if g != 'Personal Indirecto' else g)
dotacion['Personal Planta'] = dotacion['Personal Planta Contrato Plazo Fijo'] + dotacion['Personal Planta Contrato Plazo Indefinido']

In [932]:
tabla = dotacion[['Tipo','Mes','Personal Planta']].pivot_table(index = 'Tipo', columns = 'Mes', aggfunc = np.sum)
tabla

Personal Planta                                        \
Mes                           1          2          3      4        5      
Tipo                                                                       
Personal Directo         1247.2600  1195.2600  1063.9000  880.2  1014.24   
Personal Indirecto        584.7333   627.8333   666.4333  544.8   656.96   

                                                            
Mes                   6       7       8       9       2021  
Tipo                                                        
Personal Directo    962.40  720.80  936.90  404.83  1477.0  
Personal Indirecto  645.13  508.03  571.17  310.53   907.0

In [933]:
tabla.loc[('Total personal')] = tabla.sum(axis=0)

In [940]:
tabla = tabla.round()
tabla

Personal Planta                                          \
Mes                           1       2       3       4       5       6      
Tipo                                                                         
Personal Directo            1247.0  1195.0  1064.0   880.0  1014.0   962.0   
Personal Indirecto           585.0   628.0   666.0   545.0   657.0   645.0   
Total personal              1832.0  1823.0  1730.0  1425.0  1671.0  1608.0   

                                                   
Mes                   7       8      9       2021  
Tipo                                               
Personal Directo     721.0   937.0  405.0  1477.0  
Personal Indirecto   508.0   571.0  311.0   907.0  
Total personal      1229.0  1508.0  715.0  2384.0